# Load Results from Previous Sessions

In [ ]:
import dill
dill.load_session('/Users/nicholasdeporzio/Desktop/cfworkspace/results/DESI_SciencePaperConfirmation/kmax_0.1h/ps4.db')

# Plot Set 1 - Match Neutrino/Relic Forecasts

# GOOD

For some cosmology, we wish to show that the fisher forecast produces identical constraints for neutrinos and for an equivalent relic.  

### OPERATIONAL NOTE: 
We want to add the same CMB information to both the Neutrino and Equivalent Relic forecasts. If I start with the same CMB Fisher matrix, where one row/column is dlogP/dM_ncdm, I must appropriately scale it before use as the CMB prior for relics by multiplying that row/column by the factor 3^(3/4). 

In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import seaborn as sns 
import cosmicfish as cf 
import matplotlib.pyplot as plt
import dill


# Instruct pyplot to use seaborn 
sns.set()

# Set project, data, CLASS directories 
projectdir = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/")
datastore = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/data.nosync1/")
classpath = os.path.join(projectdir, "class")

# Specify resolution of numerical integrals
derivative_step = 0.008 # How much to vary parameter to calculate numerical derivative
mu_integral_step = 0.05 # For calculating numerical integral wrt mu between -1 and 1 

# Generate output paths  
ps1_resultsdir = os.path.join(projectdir, 'results', 'ps1')
ps1_resultsdir_neutrino = os.path.join(ps1_resultsdir, 'neutrino')
ps1_resultsdir_relic = os.path.join(ps1_resultsdir, 'relic')
ps1_convergencedir_neutrino = os.path.join(ps1_resultsdir, 'convergence', 'neutrino')
ps1_convergencedir_relic = os.path.join(ps1_resultsdir, 'convergence', 'relic')
cf.makedirectory(ps1_resultsdir_neutrino)
cf.makedirectory(ps1_resultsdir_relic)
cf.makedirectory(ps1_convergencedir_neutrino) 
cf.makedirectory(ps1_convergencedir_relic) 

# Fiducial Cosmologies
ps1_neutrinofid = {
        "A_s" : 2.22e-9, 
        "n_s" : 0.965,
        "omega_b" : 0.02222,
        "omega_cdm" : 0.1120,
        "tau_reio" : 0.06,
        "h" : 0.70,
        "T_cmb" : 2.726, # Units [K]
        "N_ncdm" : 3., 
        "T_ncdm" : 1.95/2.726, 
        "m_ncdm" : 0.03, # Units [eV]
        "b0" : 1.0, 
        "alphak2" : 1.0,
        "sigma_fog_0" : 250000, #Units [m s^-2]
        "N_eff" : 3.046 - (3 * 1.0132), 
        "relic_fix" : None #Not used for neutrino forecasts
        } 
# For matching the neutrino/relic forecasts, we must modify temp/mass according to equation 44 
# of the Cora/Julian paper. That is: 
# m_chi = m_nu * 3^(1/4) ***where m_nu is the mass of a single neutrino in the degenerate hierarchy
# T_chi = 1.95 * 3^(1/4)
ps1_relicfid = {
        "A_s" : 2.22e-9, 
        "n_s" : 0.965,
        "omega_b" : 0.02222,
        "omega_cdm" : 0.1120,
        "tau_reio" : 0.06,
        "h" : 0.70,
        "T_cmb" : 2.726, # Units [K]
        "N_ncdm" : 1., 
        "T_ncdm" : (1.95/2.726) * np.power(3., 1./4.), # Units [T_cmb]  Are Julian's matrices at 1.64 K? 
        "m_ncdm" : 0.03 * np.power(3., 1./4.), # Units [eV]
        "b0" : 1.0, 
        "alphak2" : 1.0,
        "sigma_fog_0" : 250000, #Units [m s^-2]
        "N_eff" : 3.046 - (3 * 1.0132), 
        "relic_fix" : "T_ncdm" # Fix T_ncdm or M_ncdm 
        }

# DESI survey parameters
z_table = np.array([0.65, 0.75, 0.85, 0.95, 1.05, 1.15, 1.25, 1.35, 1.45, 1.55, 1.65, 1.75, 1.85])
dNdz = np.array([309., 2269., 1923., 2094., 1441., 1353., 1337., 523., 466., 329., 126., 0., 0.])
skycover = 14000. # Sky coverage of survey in degrees^2

#Demonstrate Convergence
#neutrinoconvergencetest = cf.convergence(
#    classpath, # Path to CLASS installation
#    datastore, # Path to directory holding CLASS output data
#    'neutrino', # 'relic' or 'neutrino' forecasting scheme 
#    ps1_neutrinofid, # The fiducial cosmology 
#    z_table, # Redshift steps in observation
#    dNdz, # Redshift noise in observation
#    fisher_order=[
#        'omega_b',                                    
#        'omega_cdm',                                  
#        'n_s',                                        
#        'A_s',                                        
#        'tau_reio',                                   
#        'h',                                          
#        'M_ncdm',                                                                  
#        'sigma_fog',                                   
#        'b0',                                         
#        'alpha_k2'
#    ],
#    fcoverage_deg=14000, # Sky coverage in observation
#    RSD=True, # Use RSD correction to Pm
#    FOG=True, # Use FOG correction to Pm
#    AP=True, # Use AP correction to PM
#    COV=True, #Use AP Change of Variables correction to PM
#    mu_step=mu_integral_step,
#    varyfactors=[0.006, 0.007, 0.008, 0.009, 0.010], # Relative factors used to compute convergence
#    showplots=True,
#    saveplots=True,
#    savepath=ps1_convergencedir_neutrino, 
#    plotparams= ['A_s',                                                          
#                'n_s',                                                          
#                'omega_b',                                                      
#                'omega_cdm',                                                    
#                'h',                                                            
#                'tau_reio',                                                     
#                'omega_ncdm',
#                'M_ncdm',
#                'sigmafog',                                                     
#                'b0',                                                           
#                'alphak2']
#    )
#neutrinoconvergencetest.gen_all_plots() # Display convergence plots
#relicconvergencetest = cf.convergence(
#    classpath, # Path to CLASS installation
#    datastore, # Path to directory holding CLASS output data
#    'relic', # 'relic' or 'neutrino' forecasting scheme 
#    ps1_relicfid, # The fiducial cosmology 
#    z_table, # Redshift steps in observation
#    dNdz, # Redshift noise in observation
#    fisher_order=[
#        'omega_b',                                    
#        'omega_cdm',                                  
#        'n_s',                                        
#        'A_s',                                        
#        'tau_reio',                                   
#        'h',                                          
#        'M_ncdm',                                                                  
#        'sigma_fog',                                   
#        'b0',                                         
#        'alpha_k2'
#    ],
#    fcoverage_deg=14000, # Sky coverage in observation
#    RSD=True, # Use RSD correction to Pm
#    FOG=True, # Use FOG correction to Pm
#    AP=True, # Use AP correction to PM
#    COV=True, #Use AP Change of Variables correction to PM
#    mu_step=mu_integral_step,
#    varyfactors=[0.006, 0.007, 0.008, 0.009, 0.010], # Relative factors used to compute convergence
#    showplots=True,
#    saveplots=True,
#    savepath=ps1_convergencedir_relic, 
#    plotparams= ['A_s',                                                          
#                'n_s',                                                          
#                'omega_b',                                                      
#                'omega_cdm',                                                    
#                'h',                                                            
#                'tau_reio',                                                     
#                'omega_ncdm',
#                'M_ncdm',
#                'sigmafog',                                                     
#                'b0',                                                           
#                'alphak2']
#    )
#relicconvergencetest.gen_all_plots() # Display convergence plots

# Run forecasts
neutrinoforecast = cf.forecast(
    classpath, 
    datastore, 
    'neutrino', 
    ps1_neutrinofid, 
    z_table, 
    "DESI", 
    dNdz, 
    fcoverage_deg=skycover, 
    dstep=derivative_step,
    RSD=True,
    FOG=True,
    AP=True,
    COV=True)
neutrinoforecast.gen_pm()
neutrinoforecast.gen_fisher(
    fisher_order=[
        'omega_b',                                    
        'omega_cdm',                                  
        'n_s',                                        
        'A_s',                                        
        'tau_reio',                                   
        'h',                                                                           
        'omega_ncdm',                                 
        'sigma_fog',                                   
        'b0',                                         
        'alpha_k2'],
    mu_step=mu_integral_step, 
    skipgen=False)
relicforecast = cf.forecast(
    classpath, 
    datastore, 
    'relic', 
    ps1_relicfid, 
    z_table,
    "DESI",
    dNdz, 
    fcoverage_deg=skycover, 
    dstep=derivative_step,
    RSD=True,
    FOG=True,
    AP=True,
    COV=True)
relicforecast.gen_pm()
relicforecast.gen_fisher(
    fisher_order=[
        'omega_b',                                    
        'omega_cdm',                                  
        'n_s',                                        
        'A_s',                                        
        'tau_reio',                                   
        'h',                                                                             
        'omega_ncdm',
        'sigma_fog',                                   
        'b0',                                         
        'alpha_k2'],
    mu_step=mu_integral_step, 
    skipgen=False)

dill.dump_session(os.path.join(ps1_resultsdir, 'ps1.db'))

In [ ]:
import dill
dill.load_session('/Users/nicholasdeporzio/Desktop/cfworkspace/results/ps1/ps1.db')

# Add CMB data
neutrinoforecast.load_cmb_fisher(
    fisher_order=[
        'omega_b',                                    
        'omega_cdm',                                  
        'n_s',                                        
        'A_s',                                        
        'tau_reio',                                   
        'h',                                                                             
        'omega_ncdm'],
    fisherpath=os.path.join(cf.priors_directory(), "CMBS4_Fisher_Neutrinos.dat"),
    )

relicforecast.load_cmb_fisher( # We are specifically adding the CMBS4 fisher information for neutrinos
    fisher_order=[             # for the purpose of making a comparison, though we should add the
        'omega_b',             # information for relics in the real forecast                          
        'omega_cdm',                                  
        'n_s',                                        
        'A_s',                                        
        'tau_reio',                                   
        'h',                                                                             
        'omega_ncdm'],
    fisherpath=os.path.join(cf.priors_directory(), "CMBS4_Fisher_Relics.dat"),
    )

# Save results
neutrinoforecast.export_matrices(ps1_resultsdir_neutrino)
relicforecast.export_matrices(ps1_resultsdir_relic)

In [ ]:
# MAKE PLOTS
# Show plots inline, and load main getdist plot module and samples class
from __future__ import print_function
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import sys, os
sys.path.insert(0,os.path.realpath(os.path.join(os.getcwd(),'..')))
from getdist import plots, MCSamples, loadMCSamples
from getdist.gaussian_mixtures import GaussianND
import getdist
# use this *after* importing getdist if you want to use interactive plots
# %matplotlib notebook
import matplotlib.pyplot as plt
import IPython
import numpy as np
import pandas as pd
print('GetDist Version: %s, Matplotlib version: %s'%(getdist.__version__, plt.matplotlib.__version__))
# matplotlib 2 may not work very well without usetex on, can uncomment
# plt.rcParams['text.usetex']=True

#paramnames = [r'$\omega_b$', r'$\omega_{cdm}$', r'$n_s$', r'$A_s$', r'$\tau_{reio}$', 'h', r'$\omega_{ncdm}$', r'$\sigma_{fog}$', r'$\bar{b}_0$', r'$\alpha_{k2}$']
#fiducials = [0.02222, 0.1120, 0.965, 2.22e-9, 0.06, 0.7, 0.09/93.14, 250000, 1.0, 1.0]
paramnames = [r'$\omega_b$', r'$\omega_{cdm}$', r'$n_s$', r'$A_s$', r'$\tau_{reio}$', 'h', r'$\omega_\chi$']
fiducials = [0.02222, 0.1120, 0.965, 2.22e-9, 0.06, 0.7, 0.09/93.14]

reliccovmat = np.loadtxt("/Users/nicholasdeporzio/Desktop/cfworkspace/results/ps1/relic/inv_cmbfisher.mat", skiprows=1)
neutrinocovmat = np.loadtxt("/Users/nicholasdeporzio/Desktop/cfworkspace/results/ps1/neutrino/inv_cmbfisher.mat", skiprows=1)

relicgauss = GaussianND(fiducials, reliccovmat, names=paramnames)
neutrinogauss = GaussianND(fiducials, neutrinocovmat, names=paramnames)

g = plots.get_subplot_plotter()
g.settings.axes_fontsize=20
g.settings.axes_labelsize=28
g.settings.axis_tick_x_rotation=45
g.settings.legend_fontsize=28
g.triangle_plot(
    [relicgauss, neutrinogauss], 
    #filled=True,
    legend_labels=['Equivalent Relic', 'Neutrino'],
    legend_loc='upper right',
    line_args = [
        {'ls':'dotted', 'lw':2, 'color':'#A51C30'},
        {'ls':'--', 'lw':2, 'color':'#93a1ad'}],
    contour_args=[
        {'ls':'dotted', 'lw':2, 'color':'#A51C30', 'filled' : False},
        {'ls':'--', 'lw':2, 'color':'#93a1ad', 'filled' : False}])
#plt.suptitle('Note: Same CMB fisher used for both cases.', fontsize=16)
plt.savefig("/Users/nicholasdeporzio/Desktop/cfworkspace/results/ps1/relicneutrino_match.png")

# Plot Set 2 - Compare Neutrino Forecast with MCMC for DESI

We wish to generate a LCDM forecast for the DESI 2D posteriors overplotted on MCMC contours... We should forecast for:
1. relic steps on  
2. relic steps off 
3. shift in parameter values due to relics steps on/off 
4. overplot (1)-(3) on MCMC contours 

In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import seaborn as sns 
import cosmicfish as cf 
import matplotlib.pyplot as plt
import dill

print(cf.RSD_DELTA_L_NUMERATOR_FACTOR)
print(cf.RSD_DELTA_LAMBDACDM)


# Instruct pyplot to use seaborn 
sns.set()

# Set project, data, CLASS directories 
projectdir = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/")
datastore = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/data.nosync2/")
classpath = os.path.join(projectdir, "class")

# Specify resolution of numerical integrals
derivative_step = 0.008 # How much to vary parameter to calculate numerical derivative
mu_integral_step = 0.05 # For calculating numerical integral wrt mu between -1 and 1 

# Generate output paths  
ps2_resultsdir = os.path.join(projectdir, 'results', 'ps2')
ps2_convergencedir = os.path.join(ps2_resultsdir, 'convergence')
cf.makedirectory(ps2_resultsdir)
cf.makedirectory(ps2_convergencedir) 


# No Relicfast CMBS4_desi_DH 2020-01-30 Chains Bestfit 
#ps2_fid = { 
#        "A_s" : 2.234591e-09, 
#        "n_s" : 9.659302e-01,
#        "omega_b" : 2.227047e-02,
#        "omega_cdm" : 1.127401e-01,
#        "tau_reio" : 6.019503e-02,
#        "h" : 7.007968e-01,
#        "T_cmb" : 2.726, # Units [K]
#        "N_ncdm" : 3., 
#        "T_ncdm" : 1.95/2.726, 
#        "m_ncdm" : 3.623808e-02, # Units [eV]
#        "b0" : 1.003223e+00, 
#        "alphak2" : 9.948011e-01,
#        "sigma_fog_0" : 2.497568e+05, #Units [m s^-2]
#        "N_eff" : 3.046 - (3 * 1.0132), 
#        "relic_fix" : None, #Not used for neutrino forecasts
#        }

# No Relicfast CMBS4_desi_DH 2020-01-30 Chains Bestfit PLUS Forecasted Shift for relicfast
ps2_fid = { 
        "A_s" : 2.234591e-09-4.806938031745625e-12, 
        "n_s" : 9.659302e-01+5.311035111640709e-05,
        "omega_b" : 2.227047e-02+1.1602668787041021e-06,
        "omega_cdm" : 1.127401e-01-4.108796815856762e-05,
        "tau_reio" : 6.019503e-02-0.001064158458315645,
        "h" : 7.007968e-01+0.0010625065009387986,
        "T_cmb" : 2.726, # Units [K]
        "N_ncdm" : 3., 
        "T_ncdm" : 1.95/2.726, 
        "m_ncdm" : 3.623808e-02-(0.009823889445796/3), # Units [eV]
        "b0" : 1.003223e-01-0.004980685985104448, 
        "alphak2" : 9.948011e+00+0.10341895410597579,
        "sigma_fog_0" : 2.497568e+05+762.6869526527007, #Units [m s^-2]
        "N_eff" : 3.046 - (3 * 1.0132), 
        "relic_fix" : None, #Not used for neutrino forecasts
        }


# Yes Relicfast CMBS4_desi_DH 2020-01-30 Chains Bestfit 
#ps2_fid = { 
#        "A_s" : 2.240779e-09, 
#        "n_s" : 9.670359e-01,
#        "omega_b" : 2.224689e-02,
#        "omega_cdm" : 1.124890e-01,
#        "tau_reio" : 6.196884e-02,
#        "h" : 7.014706e-01,
#        "T_cmb" : 2.726, # Units [K]
#        "N_ncdm" : 3., 
#        "T_ncdm" : 1.95/2.726, 
#        "m_ncdm" : 3.639409e-02, # Units [eV]
#        "b0" : 9.993837e-01, 
#        "alphak2" : 1.042483e+00,
#        "sigma_fog_0" : 2.502574e+05, #Units [m s^-2]
#        "N_eff" : 3.046 - (3 * 1.0132), 
#        "relic_fix" : None, #Not used for neutrino forecasts
#        }

# DESI survey parameters
z_table = np.array([0.65, 0.75, 0.85, 0.95, 1.05, 1.15, 1.25, 1.35, 1.45, 1.55, 1.65, 1.75, 1.85])
dNdz = np.array([309., 2269., 1923., 2094., 1441., 1353., 1337., 523., 466., 329., 126., 0., 0.])
skycover = 14000. # Sky coverage of survey in degrees^2

#Demonstrate Convergence
#ps2_convergencetest = cf.convergence(
#    classpath, # Path to CLASS installation
#    datastore, # Path to directory holding CLASS output data
#    'neutrino', # 'relic' or 'neutrino' forecasting scheme 
#    ps2_fid, # The fiducial cosmology 
#    z_table, # Redshift steps in observation
#    dNdz, # Redshift noise in observation
#    fisher_order=[
#        'omega_b',                                    
#        'omega_cdm',                                  
#        'n_s',                                        
#        'A_s',                                        
#        'tau_reio',                                   
#        'h',                                          
#        'M_ncdm',                                                                  
#        'sigma_fog',                                   
#        'b0',                                         
#        'alpha_k2'
#    ],
#    fcoverage_deg=14000, # Sky coverage in observation
#    RSD=True, # Use RSD correction to Pm
#    FOG=True, # Use FOG correction to Pm
#    AP=True, # Use AP correction to PM
#    COV=True, #Use AP Change of Variables correction to PM
#    mu_step=mu_integral_step,
#    varyfactors=[0.006, 0.007, 0.008, 0.009, 0.010], # Relative factors used to compute convergence
#    showplots=True,
#    saveplots=True,
#    savepath=ps2_convergencedir, 
#    plotparams= ['A_s',                                                          
#                'n_s',                                                          
#                'omega_b',                                                      
#                'omega_cdm',                                                    
#                'h',                                                            
#                'tau_reio',                                                     
#                'omega_ncdm',
#                'M_ncdm',
#                'sigmafog',                                                     
#                'b0',                                                           
#                'alphak2'] 
#    )
#ps2_convergencetest.gen_all_plots() # Display convergence plots

# Run Fisher Forecast for cosmology
ps2_forecast = cf.forecast(
    classpath, 
    datastore, 
    'neutrino', 
    ps2_fid, 
    z_table, 
    "DESI",
    dNdz, 
    fcoverage_deg=skycover,
    dstep=derivative_step,
    RSD=True,
    FOG=True,
    AP=True,
    COV=True)
ps2_forecast.gen_pm()
ps2_forecast.gen_fisher(
    fisher_order=[
        'omega_b',                                    
        'omega_cdm',                                  
        'n_s',                                        
        'A_s',                                        
        'tau_reio',                                   
        'h',                                          
        'M_ncdm',                                                                  
        'sigma_fog',                                   
        'b0',                                         
        'alpha_k2'
    ],
    mu_step=mu_integral_step, 
    skipgen=False)
dill.dump_session(os.path.join(ps2_resultsdir, 'ps2.db'))

In [ ]:
#import dill
#dill.load_session('/Users/nicholasdeporzio/Desktop/cfworkspace/results/ps2/ps2.db')

# Add CMB information
ps2_forecast.load_cmb_fisher(
        fisher_order=[
            'omega_b',                                    
            'omega_cdm',                                  
            'n_s',                                        
            'A_s',                                        
            'tau_reio',                                   
            'h',                                                                             
            'M_ncdm'],
        fisherpath=os.path.join(cf.priors_directory(), "CMBS4_Fisher_Neutrinos.dat"), 
        colnames=[
            'omega_b',                                    
            'omega_cdm',                                  
            'n_s',                                        
            'A_s',                                        
            'tau_reio',                                   
            'h',                                                                             
            'M_ncdm'])

# Save results
ps2_forecast.export_matrices(ps2_resultsdir)


# Print errors
for i in range(10):
    print(ps2_forecast.fisher_order[i], "uncertainty: ", np.sqrt(ps2_forecast.numpy_full_covariance[i, i]))

# Plot Set 3 - Compare Neutrino Forecast with MCMC for EUCLID

We wish to generate a LCDM forecast for the EUCLID 2D posteriors overplotted on MCMC contours... We should forecast for:
1. relic steps on  
2. relic steps off 
3. shift in parameter values due to relics steps on/off 
4. overplot (1)-(3) on MCMC contours 

In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import seaborn as sns 
import cosmicfish as cf 
import matplotlib.pyplot as plt
import dill


# Instruct pyplot to use seaborn 
sns.set()

# Set project, data, CLASS directories 
projectdir = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/")
datastore = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/data.nosync3/")
classpath = os.path.join(projectdir, "class")

# Specify resolution of numerical integrals
derivative_step = 0.008 # How much to vary parameter to calculate numerical derivative
mu_integral_step = 0.05 # For calculating numerical integral wrt mu between -1 and 1 

# Generate output paths  
ps3_resultsdir = os.path.join(projectdir, 'results', 'ps3')
ps3_convergencedir = os.path.join(ps3_resultsdir, 'convergence')
cf.makedirectory(ps3_resultsdir)
cf.makedirectory(ps3_convergencedir) 

#MCMC Fiducial
# Fiducial used for MCMCs
#ps3_fid = { 
#        "A_s" : 2.22e-9, 
#        "n_s" : 0.965,
#        "omega_b" : 0.02222,
#        "omega_cdm" : 0.1120,
#        "tau_reio" : 0.06,
#        "h" : 0.70,
#        "T_cmb" : 2.726, # Units [K]
#        "N_ncdm" : 3., 
#        "T_ncdm" : 1.95/2.726, 
#        "m_ncdm" : 0.03, # Units [eV]
#        "b0" : 1.0, 
#        "beta0" : 1.7, 
#        "beta1" : 0.5,
#        "alphak2" : 1.0,
#        "sigma_fog_0" : 250000, #Units [m s^-2]
#        "N_eff" : 3.046 - (3 * 1.0132), 
#        "relic_fix" : None, #Not used for neutrino forecasts
#        }


# EUCLID survey parameters
z_table = np.array([0.65, 0.75, 0.85, 0.95, 1.05, 1.15, 1.25, 1.35, 1.45, 1.55, 1.65, 1.75, 1.85, 1.95])
dNdz = np.array([2434.280, 4364.812, 4728.559, 4825.798, 4728.797, 4507.625, 4269.851, 3720.657, 3104.309, 
    2308.975, 1514.831, 1474.707, 893.716, 497.613])
skycover = 0.3636 # Sky coverage of survey in fraction

#Demonstrate Convergence
#ps3_convergencetest = cf.convergence(
#    classpath, # Path to CLASS installation
#    datastore, # Path to directory holding CLASS output data
#    'neutrino', # 'relic' or 'neutrino' forecasting scheme 
#    ps3_fid, # The fiducial cosmology 
#    z_table, # Redshift steps in observation
#    dNdz, # Redshift noise in observation
#    fisher_order=[
#        'omega_b',                                    
#        'omega_cdm',                                  
#        'n_s',                                        
#        'A_s',                                        
#        'tau_reio',                                   
#        'h',                                          
#        'M_ncdm',                                                                  
#        'sigma_fog',                                   
#        'b0',                                         
#        'alpha_k2'
#    ],
#    fsky=skycover, # Sky coverage in observation
#    RSD=True, # Use RSD correction to Pm
#    FOG=True, # Use FOG correction to Pm
#    AP=True, # Use AP correction to PM
#    COV=True, #Use AP Change of Variables correction to PM
#    mu_step=mu_integral_step,
#    varyfactors=[0.006, 0.007, 0.008, 0.009, 0.010], # Relative factors used to compute convergence
#    showplots=True,
#    saveplots=True,
#    savepath=ps3_convergencedir, 
#    plotparams= ['A_s',                                                          
#                'n_s',                                                          
#                'omega_b',                                                      
#                'omega_cdm',                                                    
#                'h',                                                            
#                'tau_reio',                                                     
#                'omega_ncdm',
#                'M_ncdm',
#                'sigmafog',                                                     
#                'b0',                                                           
#                'alphak2'] 
#    )
#ps3_convergencetest.gen_all_plots() # Display convergence plots

# Run Fisher Forecast
ps3_forecast = cf.forecast(
    classpath, 
    datastore, 
    'neutrino', 
    ps3_fid, 
    z_table, 
    "EUCLID", 
    dNdz, 
    fsky=skycover,
    dstep=derivative_step,
    RSD=True,
    FOG=True,
    AP=True,
    COV=True)
ps3_forecast.gen_pm()
ps3_forecast.gen_fisher(
    fisher_order=[
        'omega_b',                                    
        'omega_cdm',                                  
        'n_s',                                        
        'A_s',                                        
        'tau_reio',                                   
        'h',                                          
        'M_ncdm',                                                                    
        'sigma_fog',                                   
        'beta0',
        'beta1',                                       
        'alpha_k2'
    ],
    mu_step=mu_integral_step, 
    skipgen=False)

dill.dump_session(os.path.join(ps3_resultsdir, 'ps3.db'))

# Add CMB information
ps3_forecast.load_cmb_fisher(
    fisher_order=[
        'omega_b',                                    
        'omega_cdm',                                  
        'n_s',                                        
        'A_s',                                        
        'tau_reio',                                   
        'h',                                                                             
        'M_ncdm'],
    fisherpath=os.path.join(cf.priors_directory(), "CMBS4_Fisher_Neutrinos.dat"),
    colnames=[
        'omega_b',                                    
        'omega_cdm',                                  
        'n_s',                                        
        'A_s',                                        
        'tau_reio',                                   
        'H_0',                                                                             
        'M_ncdm'])

# Save results
ps3_forecast.export_matrices(ps3_resultsdir)

# Print errors
for i in range(10):
    print(ps3_forecast.fisher_order[i], "uncertainty: ", np.sqrt(ps3_forecast.numpy_full_covariance[i, i]))

In [ ]:
import dill
dill.load_session("/Users/nicholasdeporzio/Desktop/cfworkspace/results/NeutrinoMatch_CMBS4_euclid_beta_3_DH/ps3_CMBS4_norelicfast_beta3bestfit/ps3.db")

In [ ]:
ps3_fid

# Plot Set 4 - Relic Forecast for DESI at 1.5072 [K] 

We wish to plot the uncertainty in relic temperature as a function of relic mass for masses on the range of 10^-1 to 10^2 [eV] at fiducial relic temperature 1.5072 [K] with the DESI survey. 

In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import seaborn as sns 
import cosmicfish as cf 
import matplotlib.pyplot as plt
import dill


# Instruct pyplot to use seaborn 
sns.set()

# Set project, data, CLASS directories 
projectdir = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/")
datastore = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/data.nosync4/")
classpath = os.path.join(projectdir, "class")

# Specify resolution of numerical integrals
derivative_step = 0.008 # How much to vary parameter to calculate numerical derivative
mu_integral_step = 0.05 # For calculating numerical integral wrt mu between -1 and 1 


# Generate output paths  
ps4_resultsdir = os.path.join(projectdir, 'results', 'ps4')
ps4_convergencedir = os.path.join(ps4_resultsdir, 'convergence')
cf.makedirectory(ps4_resultsdir)
cf.makedirectory(ps4_convergencedir) 

# Linda Fiducial Cosmology 
ps4_fid = {
        "A_s" : 2.2321e-9, 
        "n_s" : 0.967,
        "omega_b" : 0.02226,
        "omega_cdm" : 0.1127,
        "tau_reio" : 0.0598,
        "h" : 0.701,
        "T_cmb" : 2.726, # Units [K]
        "N_ncdm" : 1., 
        "T_ncdm" : (1.5072/2.726), # Units [T_cmb]. 
        "m_ncdm" : 0.0328, # Units [eV]
        "b0" : 1.0, 
        "alphak2" : 1.0,
        "sigma_fog_0" : 250000, #Units [m s^-2]
        "N_eff" : 3.046, #We allow relativistic neutrinos in addition to our DM relic
        "relic_fix" : "m_ncdm" # Fix T_ncdm or m_ncdm 
        }

# DESI Parameters
z_table = z_table = np.array([0.65, 0.75, 0.85, 0.95, 1.05, 1.15, 1.25, 1.35, 1.45, 1.55, 1.65, 1.75, 1.85])
dNdz = np.array([309., 2269., 1923., 2094., 1441., 1353., 1337., 523., 466., 329., 126., 0., 0.])
skycover = 14000. # Sky coverage of survey in degrees^2

# Demonstrate Convergence
#ps4_convergencetest = cf.convergence(
#    classpath, # Path to CLASS installation
#    datastore, # Path to directory holding CLASS output data
#    'relic', # 'relic' or 'neutrino' forecasting scheme 
#    ps4_fid, # The fiducial cosmology 
#    z_table, # Redshift steps in observation
#    dNdz, # Redshift noise in observation
#    fisher_order=[
#        'omega_b',                                    
#        'omega_cdm',                                  
#        'n_s',                                        
#        'A_s',                                        
#        'tau_reio',                                   
#        'h',                                          
#        'T_ncdm', 
#        'sigma_fog',                                   
#        'b0',                                         
#        'alpha_k2'
#    ],
#    fcoverage_deg=skycover, # Sky coverage in observation
#    RSD=True, # Use RSD correction to Pm
#    FOG=True, # Use FOG correction to Pm
#    AP=True, # Use AP correction to PM
#    COV=True, #Use AP Change of Variables correction to PM
#    mu_step=mu_integral_step,
#    varyfactors=[0.006, 0.007, 0.008, 0.009, 0.010], # Relative factors used to compute convergence
#    showplots=True,
#    saveplots=True,
#    savepath=ps4_convergencedir, 
#    plotparams= ['A_s',                                                          
#                'n_s',                                                          
#                'omega_b',                                                      
#                'omega_cdm',                                                    
#                'h',                                                            
#                'tau_reio',                                                     
#                'omega_ncdm',
#                'M_ncdm',
#                'sigmafog',                                                     
#                'b0',                                                           
#                'alphak2'] 
#    )
#ps4_convergencetest.gen_all_plots()

# Run Fisher Forecast
#masses = np.geomspace(0.1, 22.0, 25)
masses = np.append(np.array([0.001, 0.01]), np.geomspace(0.1, 22.0, 25))

omegacdm_set = ps4_fid['omega_cdm'] - ((masses/cf.NEUTRINO_SCALE_FACTOR)*np.power(ps4_fid['T_ncdm']*2.726 / 1.95, 3.))                                     
ps4_fiducialset = [dict(ps4_fid, **{'m_ncdm' : masses[midx], 'omega_cdm' : omegacdm_set[midx]}) 
               for midx, mval in enumerate(masses)]
ps4_forecastset = [cf.forecast(
    classpath, 
    datastore, 
    'relic', 
    fidval, 
    z_table, 
    "DESI",
    dNdz, 
    fcoverage_deg=skycover, 
    dstep=derivative_step,
    RSD=True,
    FOG=True,
    AP=True,
    COV=True) for fididx, fidval in enumerate(ps4_fiducialset)]
for fidx, fcst in enumerate(ps4_forecastset): 
    fcst.gen_pm()
    fcst.gen_fisher(
        fisher_order=[
            'omega_b',                                    
            'omega_cdm',                                  
            'n_s',                                        
            'A_s',                                        
            'tau_reio',                                   
            'h',                                                                             
            'T_ncdm',                                 
            'sigma_fog',                                   
            'b0',                                         
            'alpha_k2'],
        mu_step=mu_integral_step, 
        skipgen=False)
    print("Relic Forecast ", fidx, " complete...")

In [ ]:
dill.dump_session(os.path.join(ps4_resultsdir, 'ps4.db'))

In [ ]:
import dill
dill.load_session('/Users/nicholasdeporzio/Desktop/cfworkspace/results/PRESENT_results_fixedkmaxdynamickmin_CMBS4_DesiPlanckProperParameterization_BossParameterizedAsEuclid/ps4/ps4.db')

# Save results 
#inpath = "/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/cosmicfish/priors/New_Relic_CMB_Fisher_Matrices/FisherCMBS4_bin"
inpath = "/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/cosmicfish/priors/New_Relic_CMB_Fisher_Matrices/FisherPlanck_bin"
head = 'omega_b \t omega_cdm \t n_s \t A_s \t tau_reio \t h \t T_ncdm \t sigma_fog \t beta0 \t beta1 \t alpha_k2'
for fidx, fval in enumerate(ps4_forecastset):
    fval.load_cmb_fisher(
        fisher_order=[
            'omega_b',                                    
            'omega_cdm',                                  
            'n_s',                                        
            'A_s',                                        
            'tau_reio',                                   
            'h',                                                                             
            'T_ncdm'],
        fisherpath=inpath+str(fidx+1)+"_T1.5K.txt", 
        colnames=[
            'omega_b',                                    
            'omega_cdm',                                  
            'n_s',                                        
            'A_s',                                        
            'tau_reio',                                   
            'h',                                                                             
            'T_ncdm[gamma]'])
    print("CMB Fisher information loaded to forecast " + str(fidx) + "...")
    outdir=os.path.join(ps4_resultsdir, 'M'+str(fidx+1))
    cf.makedirectory(outdir)
    fval.export_matrices(outdir)
    np.savetxt(outdir + '_Full_Fisher.dat', fval.numpy_full_fisher, delimiter='\t', header=head)

# Plot Set 5 - Relic Forecast for DESI at 0.9052 [K]

We wish to plot the uncertainty in relic temperature as a function of relic mass for masses on the range of 10^-1 to 10^2 [eV] at fiducial relic temperature 0.9052 [K] with the  DESI  survey. 

In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import seaborn as sns 
import cosmicfish as cf 
import matplotlib.pyplot as plt
import dill


# Instruct pyplot to use seaborn 
sns.set()

# Set project, data, CLASS directories 
projectdir = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/")
datastore = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/data.nosync5/")
classpath = os.path.join(projectdir, "class")

# Specify resolution of numerical integrals
derivative_step = 0.008 # How much to vary parameter to calculate numerical derivative
mu_integral_step = 0.05 # For calculating numerical integral wrt mu between -1 and 1 

# Generate output paths  
ps5_resultsdir = os.path.join(projectdir, 'results', 'ps5')
ps5_convergencedir = os.path.join(ps5_resultsdir, 'convergence')
cf.makedirectory(ps5_resultsdir)
cf.makedirectory(ps5_convergencedir) 

# Set fiducial cosmology 
# Linda Fiducial Cosmology 
ps5_fid = {
        "A_s" : 2.2321e-9, 
        "n_s" : 0.967,
        "omega_b" : 0.02226,
        "omega_cdm" : 0.1127,
        "tau_reio" : 0.0598,
        "h" : 0.701,
        "T_cmb" : 2.726, # Units [K]
        "N_ncdm" : 1., 
        "T_ncdm" : (0.9052/2.726), # Units [T_cmb]. 
        "m_ncdm" : 0.0328, # Units [eV]
        "b0" : 1.0, 
        "alphak2" : 1.0,
        "sigma_fog_0" : 250000, #Units [m s^-2]
        "N_eff" : 3.046, #We allow relativistic neutrinos in addition to our DM relic
        "relic_fix" : "m_ncdm" # Fix T_ncdm or m_ncdm 
        }

# DESI Parameters
z_table = z_table = np.array([0.65, 0.75, 0.85, 0.95, 1.05, 1.15, 1.25, 1.35, 1.45, 1.55, 1.65, 1.75, 1.85])
dNdz = np.array([309., 2269., 1923., 2094., 1441., 1353., 1337., 523., 466., 329., 126., 0., 0.])
skycover = 14000. # Sky coverage of survey in degrees^2

# Demonstrate Convergence
#ps5_convergencetest = cf.convergence(
#   classpath, # Path to CLASS installation
#    datastore, # Path to directory holding CLASS output data
#    'relic', # 'relic' or 'neutrino' forecasting scheme 
#    ps5_fid, # The fiducial cosmology 
#    z_table, # Redshift steps in observation
#    dNdz, # Redshift noise in observation
#    fisher_order=[
#        'omega_b',                                    
#        'omega_cdm',                                  
#        'n_s',                                        
#        'A_s',                                        
#        'tau_reio',                                   
#        'h',                                          
#        'T_ncdm', 
#        'sigma_fog',                                   
#        'b0',                                         
#        'alpha_k2'
#    ],
#    fcoverage_deg=skycover, # Sky coverage in observation
#    RSD=True, # Use RSD correction to Pm
#    FOG=True, # Use FOG correction to Pm
#    AP=True, # Use AP correction to PM
#    COV=True, #Use AP Change of Variables correction to PM
#    mu_step=mu_integral_step,
#    varyfactors=[0.006, 0.007, 0.008, 0.009, 0.010], # Relative factors used to compute convergence
#    showplots=True,
#    saveplots=True,
#    savepath=ps5_convergencedir, 
#    plotparams= ['A_s',                                                          
#                'n_s',                                                          
#                'omega_b',                                                      
#                'omega_cdm',                                                    
#                'h',                                                            
#                'tau_reio',                                                     
#                'omega_ncdm',
#                'M_ncdm',
#                'sigmafog',                                                     
#                'b0',                                                           
#                'alphak2'] 
#    )
#ps5_convergencetest.gen_all_plots()

# Run Fisher Forecast
masses = np.append(np.array([0.001, 0.01]), np.geomspace(0.1, 22.0, 25))

omegacdm_set = ps5_fid['omega_cdm'] - ((masses/cf.NEUTRINO_SCALE_FACTOR)*np.power(ps5_fid['T_ncdm']*2.726 / 1.95, 3.))                                     
ps5_fiducialset = [dict(ps5_fid, **{'m_ncdm' : masses[midx], 'omega_cdm' : omegacdm_set[midx]}) 
               for midx, mval in enumerate(masses)]
ps5_forecastset = [cf.forecast(
    classpath, 
    datastore, 
    'relic', 
    fidval, 
    z_table, 
    "DESI",
    dNdz, 
    fcoverage_deg=skycover, 
    dstep=derivative_step,
    RSD=True,
    FOG=True,
    AP=True,
    COV=True) for fididx, fidval in enumerate(ps5_fiducialset)]
for fidx, fcst in enumerate(ps5_forecastset): 
    fcst.gen_pm()
    fcst.gen_fisher(
        fisher_order=[
            'omega_b',                                    
            'omega_cdm',                                  
            'n_s',                                        
            'A_s',                                        
            'tau_reio',                                   
            'h',                                                                             
            'T_ncdm',                                 
            'sigma_fog',                                   
            'b0',                                         
            'alpha_k2'],
        mu_step=mu_integral_step, 
        skipgen=False)
    print("Relic Forecast ", fidx, " complete...")

In [ ]:
dill.dump_session(os.path.join(ps5_resultsdir, 'ps5.db'))

In [ ]:
import dill
dill.load_session('/Users/nicholasdeporzio/Desktop/cfworkspace/results/PRESENT_results_fixedkmaxdynamickmin_CMBS4_DesiPlanckProperParameterization_BossParameterizedAsEuclid/ps5/ps5.db')

# Save results 
#inpath = "/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/cosmicfish/priors/New_Relic_CMB_Fisher_Matrices/FisherCMBS4_bin"
inpath = "/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/cosmicfish/priors/New_Relic_CMB_Fisher_Matrices/FisherPlanck_bin"
head = 'omega_b \t omega_cdm \t n_s \t A_s \t tau_reio \t h \t T_ncdm \t sigma_fog \t b_0 \t alpha_k2'
for fidx, fval in enumerate(ps5_forecastset):
    fval.load_cmb_fisher(
        fisher_order=[
            'omega_b',                                    
            'omega_cdm',                                  
            'n_s',                                        
            'A_s',                                        
            'tau_reio',                                   
            'h',                                                                             
            'T_ncdm'],
        fisherpath=inpath+str(fidx+1)+"_T0.9K.txt",
        colnames=[
            'omega_b',                                    
            'omega_cdm',                                  
            'n_s',                                        
            'A_s',                                        
            'tau_reio',                                   
            'h',                                                                             
            'T_ncdm[gamma]'])
    print("CMB Fisher information loaded to forecast " + str(fidx) + "...")
    outdir=os.path.join(ps5_resultsdir, 'M'+str(fidx+1))
    cf.makedirectory(outdir)
    fval.export_matrices(outdir)
    np.savetxt(outdir + 'Full_Fisher.dat', fval.numpy_full_fisher, delimiter='\t', header=head)

# Plot Set 6 - Relic Forecast for EUCLID at 1.5072 [K]

We wish to plot the uncertainty in relic temperature as a function of relic mass for masses on the range of 10^-1 to 10^2 [eV] at fiducial relic temperature 1.5072 [K] with the EUCLID survey. 

In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import seaborn as sns 
import cosmicfish as cf 
import matplotlib.pyplot as plt
import dill


# Instruct pyplot to use seaborn 
sns.set()

# Set project, data, CLASS directories 
projectdir = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/")
datastore = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/data.nosync6/")
classpath = os.path.join(projectdir, "class")

# Specify resolution of numerical integrals
derivative_step = 0.008 # How much to vary parameter to calculate numerical derivative
mu_integral_step = 0.05 # For calculating numerical integral wrt mu between -1 and 1 


# Generate output paths  
ps6_resultsdir = os.path.join(projectdir, 'results', 'ps6')
ps6_convergencedir = os.path.join(ps6_resultsdir, 'convergence')
cf.makedirectory(ps6_resultsdir)
cf.makedirectory(ps6_convergencedir) 

# Set fiducial cosmology 
# Linda Fiducial Cosmology 
ps6_fid = {
        "A_s" : 2.2321e-9, 
        "n_s" : 0.967,
        "omega_b" : 0.02226,
        "omega_cdm" : 0.1127,
        "tau_reio" : 0.0598,
        "h" : 0.701,
        "T_cmb" : 2.726, # Units [K]
        "N_ncdm" : 1., 
        "T_ncdm" : (1.5072/2.726), # Units [T_cmb]. 
        "m_ncdm" : 0.0328, # Units [eV]
        "b0" : 1.0, 
        "beta0" : 1.7, 
        "beta1" : 1.0,
        "alphak2" : 1.0,
        "sigma_fog_0" : 250000, #Units [m s^-2]
        "N_eff" : 3.046, #We allow relativistic neutrinos in addition to our DM relic
        "relic_fix" : "m_ncdm" # Fix T_ncdm or m_ncdm 
        }

# EUCLID values
z_table = np.array([0.65, 0.75, 0.85, 0.95, 1.05, 1.15, 1.25, 1.35, 1.45, 1.55, 1.65, 1.75, 1.85, 1.95])
dNdz = np.array([2434.280, 4364.812, 4728.559, 4825.798, 4728.797, 4507.625, 4269.851, 3720.657, 3104.309, 
    2308.975, 1514.831, 1474.707, 893.716, 497.613])
skycover = 0.3636 # Sky coverage of survey in fraction

# Demonstrate Convergence
#ps6_convergencetest = cf.convergence(
#    classpath, # Path to CLASS installation
#    datastore, # Path to directory holding CLASS output data
#    'relic', # 'relic' or 'neutrino' forecasting scheme 
#    ps6_fid, # The fiducial cosmology 
#    z_table, # Redshift steps in observation
#    "EUCLID",
#    dNdz, # Redshift noise in observation
#    fisher_order=[
#        'omega_b',                                    
#        'omega_cdm',                                  
#        'n_s',                                        
#        'A_s',                                        
#        'tau_reio',                                   
#        'h',                                          
#        'T_ncdm', 
#        'sigma_fog',                                   
#        'b0',                                         
#        'alpha_k2'
#    ],
#    fsky=skycover, # Sky coverage in observation
#    RSD=True, # Use RSD correction to Pm
#    FOG=True, # Use FOG correction to Pm
#    AP=True, # Use AP correction to PM
#    COV=True, #Use AP Change of Variables correction to PM
#    mu_step=mu_integral_step,
#    varyfactors=[0.006, 0.007, 0.008, 0.009, 0.010], # Relative factors used to compute convergence
#    showplots=True,
#    saveplots=True,
#    savepath=ps6_convergencedir, 
#    plotparams= ['A_s',                                                          
#                'n_s',                                                          
#                'omega_b',                                                      
#                'omega_cdm',                                                    
#                'h',                                                            
#                'tau_reio',                                                     
#                'omega_ncdm',
#                'M_ncdm',
#                'sigmafog',                                                     
#                'beta0',
#                'beta1',
#                'alphak2'] 
#    )
#ps6_convergencetest.gen_all_plots()

# Run Fisher Forecast
masses = np.append(np.array([0.001, 0.01]), np.geomspace(0.1, 22.0, 25))

omegacdm_set = ps6_fid['omega_cdm'] - ((masses/cf.NEUTRINO_SCALE_FACTOR)*np.power(ps6_fid['T_ncdm']*2.726 / 1.95, 3.))                                     
ps6_fiducialset = [dict(ps6_fid, **{'m_ncdm' : masses[midx], 'omega_cdm' : omegacdm_set[midx]}) 
               for midx, mval in enumerate(masses)]
ps6_forecastset = [cf.forecast(
    classpath, 
    datastore, 
    'relic', 
    fidval, 
    z_table, 
    "EUCLID",
    dNdz, 
    fsky=skycover, 
    dstep=derivative_step,
    RSD=True,
    FOG=True,
    AP=True,
    COV=True) for fididx, fidval in enumerate(ps6_fiducialset)]
for fidx, fcst in enumerate(ps6_forecastset): 
    fcst.gen_pm()
    fcst.gen_fisher(
        fisher_order=[
            'omega_b',                                    
            'omega_cdm',                                  
            'n_s',                                        
            'A_s',                                        
            'tau_reio',                                   
            'h',                                                                             
            'T_ncdm',                                 
            'sigma_fog',                                   
            'beta0',
            'beta1',
            'alpha_k2'],
        mu_step=mu_integral_step, 
        skipgen=False)
    print("Relic Forecast ", fidx, " complete...")

In [ ]:
dill.dump_session(os.path.join(ps6_resultsdir, 'ps6.db'))

In [ ]:
import dill
dill.load_session('/Users/nicholasdeporzio/Desktop/cfworkspace/results/PRESENT_results_fixedkmaxdynamickmin_CMBS4_DesiPlanckProperParameterization_BossParameterizedAsEuclid/ps6/ps6.db')

# Save results 
#inpath = "/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/cosmicfish/priors/New_Relic_CMB_Fisher_Matrices/FisherCMBS4_bin"
inpath = "/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/cosmicfish/priors/New_Relic_CMB_Fisher_Matrices/FisherPlanck_bin"
head = 'omega_b \t omega_cdm \t n_s \t A_s \t tau_reio \t h \t T_ncdm \t sigma_fog \t beta0 \t beta1 \t alpha_k2'
for fidx, fval in enumerate(ps6_forecastset):
    fval.load_cmb_fisher(
        fisher_order=[
            'omega_b',                                    
            'omega_cdm',                                  
            'n_s',                                        
            'A_s',                                        
            'tau_reio',                                   
            'h',                                                                             
            'T_ncdm'],
        fisherpath=inpath+str(fidx+1)+"_T1.5K.txt", 
        colnames=[
            'omega_b',                                    
            'omega_cdm',                                  
            'n_s',                                        
            'A_s',                                        
            'tau_reio',                                   
            'h',                                                                             
            'T_ncdm[gamma]'])
    print("CMB Fisher information loaded to forecast " + str(fidx) + "...")
    outdir=os.path.join(ps6_resultsdir, 'M'+str(fidx+1))
    cf.makedirectory(outdir)
    fval.export_matrices(outdir)
    np.savetxt(outdir + 'Full_Fisher.dat', fval.numpy_full_fisher, delimiter='\t', header=head)

# Plot Set 7 - Relic Forecast for EUCLID at 0.9052 [K]

We wish to plot the uncertainty in relic temperature as a function of relic mass for masses on the range of 10^-1 to 10^2 [eV] at fiducial relic temperature 0.9052 [K] with the EUCLID survey. 

In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import seaborn as sns 
import cosmicfish as cf 
import matplotlib.pyplot as plt
import dill


# Instruct pyplot to use seaborn 
sns.set()

# Set project, data, CLASS directories 
projectdir = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/")
datastore = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/data.nosync7/")
classpath = os.path.join(projectdir, "class")

# Specify resolution of numerical integrals
derivative_step = 0.008 # How much to vary parameter to calculate numerical derivative
#mu_integral_step = 0.05 # For calculating numerical integral wrt mu between -1 and 1 
mu_integral_step = 1.0

# Generate output paths  
ps7_resultsdir = os.path.join(projectdir, 'results', 'ps7')
ps7_convergencedir = os.path.join(ps7_resultsdir, 'convergence')
cf.makedirectory(ps7_resultsdir)
cf.makedirectory(ps7_convergencedir) 

# Linda Fiducial Cosmology 
ps7_fid = {
        "A_s" : 2.2321e-9, 
        "n_s" : 0.967,
        "omega_b" : 0.02226,
        "omega_cdm" : 0.1127,
        "tau_reio" : 0.0598,
        "h" : 0.701,
        "T_cmb" : 2.726, # Units [K]
        "N_ncdm" : 1., 
        "T_ncdm" : (0.9052/2.726), # Units [T_cmb]. 
        "m_ncdm" : 0.0328, # Units [eV]
        "b0" : 1.0, 
        "beta0" : 1.7, 
        "beta1" : 1.0,
        "alphak2" : 1.0,
        "sigma_fog_0" : 250000, #Units [m s^-2]
        "N_eff" : 3.046, #We allow relativistic neutrinos in addition to our DM relic
        "relic_fix" : "m_ncdm" # Fix T_ncdm or m_ncdm 
        }

# EUCLID values
z_table = np.array([0.65, 0.75, 0.85, 0.95, 1.05, 1.15, 1.25, 1.35, 1.45, 1.55, 1.65, 1.75, 1.85, 1.95])
dNdz = np.array([2434.280, 4364.812, 4728.559, 4825.798, 4728.797, 4507.625, 4269.851, 3720.657, 3104.309, 
    2308.975, 1514.831, 1474.707, 893.716, 497.613])
skycover = 0.3636 # Sky coverage of survey in fraction

#Demonstrate Convergence
#ps7_convergencetest = cf.convergence(
#    classpath, # Path to CLASS installation
#    datastore, # Path to directory holding CLASS output data
#    'relic', # 'relic' or 'neutrino' forecasting scheme 
#    ps7_fid, # The fiducial cosmology 
#    z_table, # Redshift steps in observation
#    dNdz, # Redshift noise in observation
#    fisher_order=[
#        'omega_b',                                    
#        'omega_cdm',                                  
#        'n_s',                                        
#        'A_s',                                        
#        'tau_reio',                                   
#        'h',                                          
#        'T_ncdm', 
#        'sigma_fog',                                   
#        'b0',                                         
#        'alpha_k2'
#    ],
#    fsky=skycover, # Sky coverage in observation
#    RSD=True, # Use RSD correction to Pm
#    FOG=True, # Use FOG correction to Pm
#    AP=True, # Use AP correction to PM
#    COV=True, #Use AP Change of Variables correction to PM
#    mu_step=mu_integral_step,
#    varyfactors=[0.006, 0.007, 0.008, 0.009, 0.010], # Relative factors used to compute convergence
#    showplots=True,
#    saveplots=True,
#    savepath=ps7_convergencedir, 
#    plotparams= ['A_s',                                                          
#                'n_s',                                                          
#                'omega_b',                                                      
#                'omega_cdm',                                                    
#                'h',                                                            
#                'tau_reio',                                                     
#                'omega_ncdm',
#                'M_ncdm',
#                'sigmafog',                                                     
#                'b0',                                                           
#                'alphak2'] 
#    )
#ps7_convergencetest.gen_all_plots()

# Run Fisher Forecast
masses = np.append(np.array([0.001, 0.01]), np.geomspace(0.1, 22.0, 25))

omegacdm_set = ps7_fid['omega_cdm'] - ((masses/cf.NEUTRINO_SCALE_FACTOR)*np.power(ps7_fid['T_ncdm']*2.726 / 1.95, 3.))                                     
ps7_fiducialset = [dict(ps7_fid, **{'m_ncdm' : masses[midx], 'omega_cdm' : omegacdm_set[midx]}) 
               for midx, mval in enumerate(masses)]
ps7_forecastset = [cf.forecast(
    classpath, 
    datastore, 
    'relic', 
    fidval, 
    z_table, 
    "EUCLID",
    dNdz, 
    fsky=skycover, 
    dstep=derivative_step,
    RSD=True,
    FOG=True,
    AP=True,
    COV=True) for fididx, fidval in enumerate(ps7_fiducialset)]
for fidx, fcst in enumerate(ps7_forecastset): 
    fcst.gen_pm()
    fcst.gen_fisher(
        fisher_order=[                                                          
            'omega_b',                                                          
            'omega_cdm',                                                        
            'n_s',                                                              
            'A_s',                                                              
            'tau_reio',                                                         
            'h',                                                                
            'T_ncdm',                                                           
            'sigma_fog',                                                        
            'beta0',
            'beta1',
            'alpha_k2'],
        mu_step=mu_integral_step, 
        skipgen=False
    )
    print("Relic Forecast ", fidx, " complete...")

In [ ]:
dill.dump_session(os.path.join(ps7_resultsdir, 'ps7.db'))

In [ ]:
import dill
dill.load_session('/Users/nicholasdeporzio/Desktop/cfworkspace/results/PRESENT_results_fixedkmaxdynamickmin_CMBS4_DesiPlanckProperParameterization_BossParameterizedAsEuclid/ps7/ps7.db')

In [ ]:
import cosmicfish as cf
for fidx, fcst in enumerate(ps7_forecastset): 
    fcst.reorder_fisher(
        fisher_order=[                                                          
            'omega_b',                                                          
            'omega_cdm',                                                        
            'n_s',                                                              
            'A_s',                                                              
            'tau_reio',                                                         
            'h',                                                                
            'omega_ncdm',                                                           
            'sigma_fog',                                                        
            'beta0',
            'beta1',
            'alpha_k2'])

In [ ]:

# Save results 
#inpath = "/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/cosmicfish/priors/New_Relic_CMB_Fisher_Matrices/FisherCMBS4_bin"
inpath = "/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/cosmicfish/priors/New_Relic_CMB_Fisher_Matrices/FisherPlanck_bin"
head = 'omega_b \t omega_cdm \t n_s \t A_s \t tau_reio \t h \t omega_ncdm \t sigma_fog \t beta0 \t beta1 \t alpha_k2'
for fidx, fval in enumerate(ps7_forecastset):
    fval.load_cmb_fisher(
        fisher_order=[
            'omega_b',                                    
            'omega_cdm',                                  
            'n_s',                                        
            'A_s',                                        
            'tau_reio',                                   
            'h',                                                                             
            'omega_ncdm'],
        fisherpath=inpath+str(fidx+1)+"_T0.9K.txt")
    print("CMB Fisher information loaded to forecast " + str(fidx) + "...")
    outdir=os.path.join(ps7_resultsdir, 'M'+str(fidx+1))
    cf.makedirectory(outdir)
    fval.export_matrices(outdir)
    np.savetxt(outdir + 'Full_Fisher.dat', fval.numpy_full_fisher, delimiter='\t', header=head)

# Plot Set 8 - Relic Forecast for BOSS at 1.5072[K]

In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import seaborn as sns 
import cosmicfish as cf 
import matplotlib.pyplot as plt
import dill


# Instruct pyplot to use seaborn 
sns.set()

# Set project, data, CLASS directories 
projectdir = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/")
datastore = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/data.nosync8/")
classpath = os.path.join(projectdir, "class")

# Specify resolution of numerical integrals
derivative_step = 0.008 # How much to vary parameter to calculate numerical derivative
mu_integral_step = 0.05 # For calculating numerical integral wrt mu between -1 and 1 

# Generate output paths  
ps8_resultsdir = os.path.join(projectdir, 'results', 'ps8')
ps8_convergencedir = os.path.join(ps8_resultsdir, 'convergence')
cf.makedirectory(ps8_resultsdir)
cf.makedirectory(ps8_convergencedir) 

# Set fiducial cosmology 
# Linda Fiducial Cosmology 
ps8_fid = {
        "A_s" : 2.2321e-9, 
        "n_s" : 0.967,
        "omega_b" : 0.02226,
        "omega_cdm" : 0.1127,
        "tau_reio" : 0.0598,
        "h" : 0.701,
        "T_cmb" : 2.726, # Units [K]
        "N_ncdm" : 1., 
        "T_ncdm" : (1.5072/2.726), # Units [T_cmb]. 
        "m_ncdm" : 0.0328, # Units [eV]
        "b0" : 1.0, 
        "beta0" : 1.7, 
        "beta1" : 1.0,
        "alphak2" : 1.0,
        "sigma_fog_0" : 250000, #Units [m s^-2]
        "N_eff" : 3.046, #We allow relativistic neutrinos in addition to our DM relic
        "relic_fix" : "m_ncdm" # Fix T_ncdm or m_ncdm 
        }

# BOSS values
z_table = np.array([0.05, 0.15, 0.25, 0.35, 0.45, 0.55, 0.65, 0.75])
dNdz = np.array([8., 50., 125., 222., 332., 447., 208., 30.])
skycover = 10000. # Sky coverage of BOSS in degrees^2

#Demonstrate Convergence
#ps8_convergencetest = cf.convergence(
#    classpath, # Path to CLASS installation
#    datastore, # Path to directory holding CLASS output data
#    'relic', # 'relic' or 'neutrino' forecasting scheme 
#    ps8_fid, # The fiducial cosmology 
#    z_table, # Redshift steps in observation
#    dNdz, # Redshift noise in observation
#    fisher_order=[
#        'omega_b',                                    
#        'omega_cdm',                                  
#        'n_s',                                        
#        'A_s',                                        
#        'tau_reio',                                   
#        'h',                                          
#        'T_ncdm', 
#        'sigma_fog',                                   
#        'b0',                                         
#        'alpha_k2'
#    ],
#    fsky=skycover, # Sky coverage in observation
#    RSD=True, # Use RSD correction to Pm
#    FOG=True, # Use FOG correction to Pm
#    AP=True, # Use AP correction to PM
#    COV=True, #Use AP Change of Variables correction to PM
#    mu_step=mu_integral_step,
#    varyfactors=[0.006, 0.007, 0.008, 0.009, 0.010], # Relative factors used to compute convergence
#    showplots=True,
#    saveplots=True,
#    savepath=ps8_convergencedir, 
#    plotparams= ['A_s',                                                          
#                'n_s',                                                          
#                'omega_b',                                                      
#                'omega_cdm',                                                    
#                'h',                                                            
#                'tau_reio',                                                     
#                'omega_ncdm',
#                'M_ncdm',
#                'sigmafog',                                                     
#                'b0',                                                           
#                'alphak2'] 
#    )
#ps8_convergencetest.gen_all_plots()

# Run Fisher Forecast
masses = np.append(np.array([0.001, 0.01]), np.geomspace(0.1, 22.0, 25))

omegacdm_set = ps8_fid['omega_cdm'] - ((masses/cf.NEUTRINO_SCALE_FACTOR)*np.power(ps8_fid['T_ncdm']*2.726 / 1.95, 3.))                                     
ps8_fiducialset = [dict(ps8_fid, **{'m_ncdm' : masses[midx], 'omega_cdm' : omegacdm_set[midx]}) 
               for midx, mval in enumerate(masses)]
ps8_forecastset = [cf.forecast(
    classpath, 
    datastore, 
    'relic', 
    fidval, 
    z_table, 
    "EUCLID",
    dNdz, 
    fcoverage_deg=skycover, 
    dstep=derivative_step,
    RSD=True,
    FOG=True,
    AP=True,
    COV=True) for fididx, fidval in enumerate(ps8_fiducialset)]
for fidx, fcst in enumerate(ps8_forecastset): 
    fcst.gen_pm()
    fcst.gen_fisher(
        fisher_order=[
            'omega_b',                                    
            'omega_cdm',                                  
            'n_s',                                        
            'A_s',                                        
            'tau_reio',                                   
            'h',                                                                             
            'T_ncdm',                                 
            'sigma_fog',                                   
            'beta0',
            'beta1',
            'alpha_k2'],
        mu_step=mu_integral_step, 
        skipgen=False)
    print("Relic Forecast ", fidx, " complete...")

In [ ]:
dill.dump_session(os.path.join(ps8_resultsdir, 'ps8.db'))

In [ ]:
import dill
dill.load_session('/Users/nicholasdeporzio/Desktop/cfworkspace/results/PRESENT_results_fixedkmaxdynamickmin_CMBS4_DesiPlanckProperParameterization_BossParameterizedAsEuclid/ps8/ps8.db')

# Save results 
#inpath = "/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/cosmicfish/priors/New_Relic_CMB_Fisher_Matrices/FisherCMBS4_bin"
inpath = "/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/cosmicfish/priors/New_Relic_CMB_Fisher_Matrices/FisherPlanck_bin"
head = 'omega_b \t omega_cdm \t n_s \t A_s \t tau_reio \t h \t T_ncdm \t sigma_fog \t beta0 \t beta1 \t alpha_k2'
for fidx, fval in enumerate(ps8_forecastset):
    fval.load_cmb_fisher(
        fisher_order=[
            'omega_b',                                    
            'omega_cdm',                                  
            'n_s',                                        
            'A_s',                                        
            'tau_reio',                                   
            'h',                                                                             
            'T_ncdm'],
        fisherpath=inpath+str(fidx+1)+"_T1.5K.txt", 
        colnames=[
            'omega_b',                                    
            'omega_cdm',                                  
            'n_s',                                        
            'A_s',                                        
            'tau_reio',                                   
            'h',                                                                             
            'T_ncdm[gamma]'])
    print("CMB Fisher information loaded to forecast " + str(fidx) + "...")
    outdir=os.path.join(ps8_resultsdir, 'M'+str(fidx+1))
    cf.makedirectory(outdir)
    fval.export_matrices(outdir)
    np.savetxt(outdir + 'Full_Fisher.dat', fval.numpy_full_fisher, delimiter='\t', header=head)

# Plot Set 9 - Relic Forecast for BOSS/eBOSS at 0.9052[K]

In [ ]:
import os                                                                       
import dill                                                                     
import shutil                                                                   
import numpy as np                                                              
import pandas as pd                                                             
import seaborn as sns                                                           
import cosmicfish as cf                                                         
import matplotlib.pyplot as plt                                                 
                                                                                
                                                                                
# Instruct pyplot to use seaborn                                                
sns.set()                                                                       
                                                                                
# Set project, data, CLASS directories                                          
projectdir = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/")    
datastore = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/data.nosync9/")
classpath = os.path.join(projectdir, "class")                                   
                                                                                
# Specify resolution of numerical integrals                                     
derivative_step = 0.008 # How much to vary parameter to calculate numerical derivative
mu_integral_step = 0.05 # For calculating numerical integral wrt mu between -1 and 1
                                                                                
# Generate output paths                                                         
ps9_resultsdir = os.path.join(projectdir, 'results', 'ps9')       
ps9_convergencedir = os.path.join(ps9_resultsdir, 'convergence')
cf.makedirectory(ps9_resultsdir)                                        
cf.makedirectory(ps9_convergencedir)                                    
                                                                                
#Set Fiducial cosmology                                                         
# Linda Fiducial Cosmology 
ps9_fid = {
        "A_s" : 2.2321e-9, 
        "n_s" : 0.967,
        "omega_b" : 0.02226,
        "omega_cdm" : 0.1127,
        "tau_reio" : 0.0598,
        "h" : 0.701,
        "T_cmb" : 2.726, # Units [K]
        "N_ncdm" : 1., 
        "T_ncdm" : (0.9052/2.726), # Units [T_cmb]. 
        "m_ncdm" : 0.0328, # Units [eV]
        "b0" : 1.0, 
        "beta0" : 1.7, 
        "beta1" : 1.0,
        "alphak2" : 1.0,
        "sigma_fog_0" : 250000, #Units [m s^-2]
        "N_eff" : 3.046, #We allow relativistic neutrinos in addition to our DM relic
        "relic_fix" : "m_ncdm" # Fix T_ncdm or m_ncdm 
        }

# BOSS LRGs                                                               
z_table = z_table = np.array([0.05, 0.15, 0.25, 0.35, 0.45, 0.55, 0.65, 0.75])
dNdz = np.array([8., 50., 125., 222., 332., 447., 208., 30.])    
skycover = 10000. # Sky coverage of BOSS in degrees^2
                                                                                
# Demonstrate Convergence                                                       
#ps9_convergencetest = cf.convergence(                                  
#    classpath, # Path to CLASS installation                                    
#    datastore, # Path to directory holding CLASS output data                   
#    'relic', # 'relic' or 'neutrino' forecasting scheme                        
#    ps9_fid, # The fiducial cosmology                                  
#    z_table, # Redshift steps in observation                                   
#    dNdz, # Redshift noise in observation                                      
#    fisher_order=[                                                             
#        'omega_b',                                                             
#        'omega_cdm',                                                           
#        'n_s',                                                                 
#        'A_s',                                                                 
#        'tau_reio',                                                            
#        'h',                                                                   
#        'T_ncdm',                                                              
#        'sigma_fog',                                                           
#        'b0',                                                                  
#        'alpha_k2'                                                             
#    ],                                                                         
#    fcoverage_deg=skycover, # Sky coverage in observation                      
#    RSD=True, # Use RSD correction to Pm                                       
#    FOG=True, # Use FOG correction to Pm                                       
#    AP=True, # Use AP correction to PM                                         
#    COV=True, #Use AP Change of Variables correction to PM                     
#    mu_step=mu_integral_step,                                                  
#    varyfactors=[0.006, 0.007, 0.008, 0.009, 0.010], # Relative factors used to compute convergence
#    showplots=True,                                                            
#    saveplots=True,                                                            
#    savepath=ps9_convergencedir,                                       
#    plotparams= ['A_s',                                                          
#                'n_s',                                                          
#                'omega_b',                                                      
#                'omega_cdm',                                                    
#                'h',                                                            
#                'tau_reio',                                                     
#                'omega_ncdm',                                                  
#                'M_ncdm',                                                      
#                'sigmafog',                                                     
#                'b0',                                                           
#                'alphak2']                                                     
#    )                                                                          
#ps9_convergencetest.gen_all_plots()

# Run Fisher Forecast                                                           
masses = np.append(np.array([0.001, 0.01]), np.geomspace(0.1, 22.0, 25))

omegacdm_set = ps9_fid['omega_cdm'] - ((masses/cf.NEUTRINO_SCALE_FACTOR)*np.power(ps9_fid['T_ncdm']*2.726 / 1.95, 3.))
ps9_fiducialset = [dict(ps9_fid, **{'m_ncdm' : masses[midx], 'omega_cdm' : omegacdm_set[midx]})
               for midx, mval in enumerate(masses)]                             
ps9_forecastset = [cf.forecast(                                         
    classpath,                                                                  
    datastore,                                                                  
    'relic',                                                                    
    fidval,                                                                     
    z_table, 
    "EUCLID",
    dNdz,                                                                       
    fcoverage_deg=skycover,                                                     
    dstep=derivative_step,                                                      
    RSD=True,                                                                   
    FOG=True,                                                                   
    AP=True,                                                                    
    COV=True) for fididx, fidval in enumerate(ps9_fiducialset)]         
for fidx, fcst in enumerate(ps9_forecastset):                           
    fcst.gen_pm()                                                               
    fcst.gen_fisher(                                                            
        fisher_order=[                                                          
            'omega_b',                                                          
            'omega_cdm',                                                        
            'n_s',                                                              
            'A_s',                                                              
            'tau_reio',                                                         
            'h',                                                                
            'T_ncdm',                                                           
            'sigma_fog',                                                        
            'beta0',
            'beta1',
            'alpha_k2'],                                                        
        mu_step=mu_integral_step,                                               
        skipgen=False)                                                          
    print("Relic Forecast ", fidx, " complete...")                              

In [ ]:
dill.dump_session(os.path.join(ps9_resultsdir, 'ps9.db'))

In [ ]:
import dill
dill.load_session('/Users/nicholasdeporzio/Desktop/cfworkspace/results/PRESENT_results_fixedkmaxdynamickmin_CMBS4_DesiPlanckProperParameterization_BossParameterizedAsEuclid/ps9/ps9.db')

# Save results                                                                  
#inpath = "/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/cosmicfish/priors/New_Relic_CMB_Fisher_Matrices/FisherCMBS4_bin"
inpath = "/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/cosmicfish/priors/New_Relic_CMB_Fisher_Matrices/FisherPlanck_bin"
head = 'omega_b \t omega_cdm \t n_s \t A_s \t tau_reio \t h \t T_ncdm \t sigma_fog \t beta0 \t beta1 \t alpha_k2'
for fidx, fval in enumerate(ps9_forecastset):                     
    fval.load_cmb_fisher(                                                       
        fisher_order=[                                                          
            'omega_b',                                                          
            'omega_cdm',                                                        
            'n_s',                                                              
            'A_s',                                                              
            'tau_reio',                                                         
            'h',                                                                
            'T_ncdm'],                                                          
        fisherpath=inpath+str(fidx+1)+"_T0.9K.txt",                             
        colnames=[                                                              
            'omega_b',                                                          
            'omega_cdm',                                                        
            'n_s',                                                              
            'A_s',                                                              
            'tau_reio',                                                         
            'h',                                                                
            'T_ncdm[gamma]'])                                                   
    print("CMB Fisher information loaded to forecast " + str(fidx) + "...")     
    outdir=os.path.join(ps9_resultsdir, 'M'+str(fidx+1))                  
    cf.makedirectory(outdir)                                                    
    fval.export_matrices(outdir)                                                
    np.savetxt(outdir + 'Full_Fisher.dat', fval.numpy_full_fisher, delimiter='\t', header=head)

# Plot Set 10 - Combine Constraints from Various Surveys

In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import seaborn as sns 
import cosmicfish as cf 
import matplotlib.pyplot as plt
import dill


# Instruct pyplot to use seaborn 
sns.set()

# Set project, data, CLASS directories 
projectdir = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/")
datastore = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/data.nosync10/")
classpath = os.path.join(projectdir, "class")

# Combine plots
basepath1 = '/Users/nicholasdeporzio/Desktop/cfworkspace/results/PRESENT_results_fixedkmaxdynamickmin_CMBS4_DesiPlanckProperParameterization_BossParameterizedAsEuclid/'
basepath2 = '/Users/nicholasdeporzio/Desktop/cfworkspace/results/PRESENT_results_fixedkmaxdynamickmin_PLANCK_DesiPlanckProperParameterization_BossParameterizedAsEuclid/'

# Load data
masses = np.append(np.array([0.001, 0.01]), np.geomspace(0.1, 22.0, 25))
numpts = len(masses)

mattype = '/inv_fullfisher.mat'
desi_cmbs4_1507mK_covs = [pd.read_csv(basepath1+'ps4/M'+str(idx+1)+mattype, sep='\t', header=0) for idx in range(numpts)]
desi_cmbs4_905mK_covs = [pd.read_csv(basepath1+'ps5/M'+str(idx+1)+mattype, sep='\t', header=0) for idx in range(numpts)]
euclid_cmbs4_1507mK_covs = [pd.read_csv(basepath1+'ps6/M'+str(idx+1)+mattype, sep='\t', header=0) for idx in range(numpts)]
euclid_cmbs4_905mK_covs = [pd.read_csv(basepath1+'ps7/M'+str(idx+1)+mattype, sep='\t', header=0) for idx in range(numpts)]
boss_cmbs4_1507mK_covs = [pd.read_csv(basepath1+'ps8/M'+str(idx+1)+mattype, sep='\t', header=0) for idx in range(numpts)]
boss_cmbs4_905mK_covs = [pd.read_csv(basepath1+'ps9/M'+str(idx+1)+mattype, sep='\t', header=0) for idx in range(numpts)]

desi_planck_1507mK_covs = [pd.read_csv(basepath2+'ps4/M'+str(idx+1)+mattype, sep='\t', header=0) for idx in range(numpts)]
desi_planck_905mK_covs = [pd.read_csv(basepath2+'ps5/M'+str(idx+1)+mattype, sep='\t', header=0) for idx in range(numpts)]
euclid_planck_1507mK_covs = [pd.read_csv(basepath2+'ps6/M'+str(idx+1)+mattype, sep='\t', header=0) for idx in range(numpts)]
euclid_planck_905mK_covs = [pd.read_csv(basepath2+'ps7/M'+str(idx+1)+mattype, sep='\t', header=0) for idx in range(numpts)]
boss_planck_1507mK_covs = [pd.read_csv(basepath2+'ps8/M'+str(idx+1)+mattype, sep='\t', header=0) for idx in range(numpts)]
boss_planck_905mK_covs = [pd.read_csv(basepath2+'ps9/M'+str(idx+1)+mattype, sep='\t', header=0) for idx in range(numpts)]


desi_cmbs4_1507mK_Terrs = [np.sqrt(desi_cmbs4_1507mK_covs[idx].iloc[6,6]) for idx in range(numpts)]
desi_cmbs4_905mK_Terrs = [np.sqrt(desi_cmbs4_905mK_covs[idx].iloc[6,6]) for idx in range(numpts)]
euclid_cmbs4_1507mK_Terrs = [np.sqrt(euclid_cmbs4_1507mK_covs[idx].iloc[6,6]) for idx in range(numpts)]
euclid_cmbs4_905mK_Terrs = [np.sqrt(euclid_cmbs4_905mK_covs[idx].iloc[6,6]) for idx in range(numpts)]
boss_cmbs4_1507mK_Terrs = [np.sqrt(boss_cmbs4_1507mK_covs[idx].iloc[6,6]) for idx in range(numpts)]
boss_cmbs4_905mK_Terrs = [np.sqrt(boss_cmbs4_905mK_covs[idx].iloc[6,6]) for idx in range(numpts)]

desi_planck_1507mK_Terrs = [np.sqrt(desi_planck_1507mK_covs[idx].iloc[6,6]) for idx in range(numpts)]
desi_planck_905mK_Terrs = [np.sqrt(desi_planck_905mK_covs[idx].iloc[6,6]) for idx in range(numpts)]
euclid_planck_1507mK_Terrs = [np.sqrt(euclid_planck_1507mK_covs[idx].iloc[6,6]) for idx in range(numpts)]
euclid_planck_905mK_Terrs = [np.sqrt(euclid_planck_905mK_covs[idx].iloc[6,6]) for idx in range(numpts)]
boss_planck_1507mK_Terrs = [np.sqrt(boss_planck_1507mK_covs[idx].iloc[6,6]) for idx in range(numpts)]
boss_planck_905mK_Terrs = [np.sqrt(boss_planck_905mK_covs[idx].iloc[6,6]) for idx in range(numpts)]


plotdatadir = '/Users/nicholasdeporzio/Desktop/plotdata/'
cf.makedirectory(plotdatadir)

np.savetxt(plotdatadir+'masses.csv', masses)

np.savetxt(plotdatadir+'cmbs4_desi_1507mK.csv', desi_cmbs4_1507mK_Terrs)
np.savetxt(plotdatadir+'cmbs4_desi_905mK.csv', desi_cmbs4_905mK_Terrs)
np.savetxt(plotdatadir+'cmbs4_euclid_1507mK.csv', euclid_cmbs4_1507mK_Terrs) 
np.savetxt(plotdatadir+'cmbs4_euclid_905mK.csv',  euclid_cmbs4_905mK_Terrs)
np.savetxt(plotdatadir+'cmbs4_boss_1507mK.csv', boss_cmbs4_1507mK_Terrs)
np.savetxt(plotdatadir+'cmbs4_boss_905mK.csv', boss_cmbs4_905mK_Terrs)

np.savetxt(plotdatadir+'planck_desi_1507mK.csv', desi_planck_1507mK_Terrs) 
np.savetxt(plotdatadir+'planck_desi_905mK.csv', desi_planck_905mK_Terrs)
np.savetxt(plotdatadir+'planck_euclid_1507mK.csv', euclid_planck_1507mK_Terrs)
np.savetxt(plotdatadir+'planck_euclid_905mK.csv', euclid_planck_905mK_Terrs)
np.savetxt(plotdatadir+'planck_boss_1507mK.csv', boss_planck_1507mK_Terrs)
np.savetxt(plotdatadir+'planck_boss_905mK.csv', boss_planck_905mK_Terrs)


desi_cmbs4_1507mK_Nerrs = (4. * np.power(1.5, 3.) * np.array(desi_cmbs4_1507mK_Terrs) * np.power(1.95, -4.))
desi_cmbs4_905mK_Nerrs = (4. * np.power(1.1, 3.) * np.array(desi_cmbs4_905mK_Terrs) * np.power(1.95, -4.))
euclid_cmbs4_1507mK_Nerrs = (4. * np.power(1.5, 3.) * np.array(euclid_cmbs4_1507mK_Terrs) * np.power(1.95, -4.))
euclid_cmbs4_905mK_Nerrs = (4. * np.power(1.1, 3.) * np.array(euclid_cmbs4_905mK_Terrs) * np.power(1.95, -4.))
boss_cmbs4_1507mK_Nerrs = (4. * np.power(1.5, 3.) * np.array(boss_cmbs4_1507mK_Terrs) * np.power(1.95, -4.))
boss_cmbs4_905mK_Nerrs = (4. * np.power(1.1, 3.) * np.array(boss_cmbs4_905mK_Terrs) * np.power(1.95, -4.))

desi_planck_1507mK_Nerrs = (4. * np.power(1.5, 3.) * np.array(desi_planck_1507mK_Terrs) * np.power(1.95, -4.))
desi_planck_905mK_Nerrs = (4. * np.power(1.1, 3.) * np.array(desi_planck_905mK_Terrs) * np.power(1.95, -4.))
euclid_planck_1507mK_Nerrs = (4. * np.power(1.5, 3.) * np.array(euclid_planck_1507mK_Terrs) * np.power(1.95, -4.))
euclid_planck_905mK_Nerrs = (4. * np.power(1.1, 3.) * np.array(euclid_planck_905mK_Terrs) * np.power(1.95, -4.))
boss_planck_1507mK_Nerrs = (4. * np.power(1.5, 3.) * np.array(boss_planck_1507mK_Terrs) * np.power(1.95, -4.))
boss_planck_905mK_Nerrs = (4. * np.power(1.1, 3.) * np.array(boss_planck_905mK_Terrs) * np.power(1.95, -4.))

plt.figure(figsize=(15,7.5))
plt.semilogx(masses, np.array(boss_cmbs4_1507mK_Terrs) , label="CMBS4 + BOSS", linestyle='dashed', color='red')
plt.semilogx(masses, np.array(desi_cmbs4_1507mK_Terrs), label="CMBS4 + DESI", linestyle='dashed', color='green')
plt.semilogx(masses, np.array(euclid_cmbs4_1507mK_Terrs) , label="CMBS4 + EUCLID", linestyle='dashed', color='blue')
plt.semilogx(masses, np.array(boss_planck_1507mK_Terrs) , label="Planck + BOSS", linestyle='solid', color='red')
plt.semilogx(masses, np.array(desi_planck_1507mK_Terrs), label="Planck + DESI", linestyle='solid', color='green')
plt.semilogx(masses, np.array(euclid_planck_1507mK_Terrs) , label="Planck + EUCLID", linestyle='solid', color='blue')
plt.semilogx(masses, numpts*[1.5072/5], label=r"$T_{\chi, 0}/5$", linestyle='solid', color='black')

plt.title(r"$T_{\chi, 0}$ = 1.5 K", fontsize=24)
plt.xlabel(r"$M_\chi$[eV]", fontsize=24)
plt.ylabel(r"$\sigma_{T_\chi}$", fontsize=24)
plt.legend(fontsize=24)
plt.tick_params(axis='x', which='minttor')
plt.grid(True, which='minor')
plt.savefig("/Users/nicholasdeporzio/Desktop/relicforecast_T1507mk.png")
plt.show()

##
cutmasses=np.delete(masses, 3)
cut905mKTerrboss = np.delete(boss_planck_905mK_Terrs, 3)
cut905mKTerrdesi = np.delete(desi_planck_905mK_Terrs, 3)
cut905mKTerreuclid = np.delete(euclid_planck_905mK_Terrs, 3)
##

plt.figure(figsize=(15,7.5))
plt.semilogx(masses, np.array(boss_cmbs4_905mK_Terrs) , label="CMBS4 + BOSS", linestyle='dashed', color='red')
plt.semilogx(masses, np.array(desi_cmbs4_905mK_Terrs), label="CMBS4 + DESI", linestyle='dashed', color='green')
plt.semilogx(masses, np.array(euclid_cmbs4_905mK_Terrs) , label="CMBS4 + EUCLID", linestyle='dashed', color='blue')

#plt.semilogx(masses, np.array(boss_planck_905mK_Terrs) , label="BOSS, Planck, 0.9052K", linestyle='solid', color='red')
#plt.semilogx(masses, np.array(desi_planck_905mK_Terrs), label="DESI, Planck, 0.9052K", linestyle='solid', color='green')
#plt.semilogx(masses, np.array(euclid_planck_905mK_Terrs) , label="EUCLID, Planck, 0.9052K", linestyle='solid', color='blue')
##
plt.semilogx(cutmasses, np.array(cut905mKTerrboss) , label="Planck + BOSS", linestyle='solid', color='red')
plt.semilogx(cutmasses, np.array(cut905mKTerrdesi), label="Planck + DESI", linestyle='solid', color='green')
plt.semilogx(cutmasses, np.array(cut905mKTerreuclid) , label="Planck + EUCLID, Planck", linestyle='solid', color='blue')
##

plt.semilogx(masses, numpts*[0.9052/5], label=r"$T_{\chi, 0}/5$", linestyle='solid', color='black')

plt.title(r"$T_{\chi, 0}$ = 0.9 K", fontsize=24)
plt.xlabel(r"$M_\chi$ [eV]", fontsize=24)
plt.ylabel(r"$\sigma_{T_\chi}$", fontsize=24)
plt.legend(fontsize=24)
plt.tick_params(axis='x', which='minttor')
plt.grid(True, which='minor')
plt.savefig("/Users/nicholasdeporzio/Desktop/relicforecast_T905mk.png")
plt.show()

In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import seaborn as sns 
import cosmicfish as cf 
import matplotlib.pyplot as plt
import dill


# Instruct pyplot to use seaborn 
sns.set()

# Set project, data, CLASS directories 
projectdir = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/")
datastore = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/data.nosync7/")
classpath = os.path.join(projectdir, "class")

# Combine plots
basepath = '/Users/nicholasdeporzio/Desktop/cfworkspace/results/'

mattype = '/inv_fullfisher.mat'
euclid_905mK_covs_1 = [pd.read_csv(basepath+'ps7/M'+str(idx+1)+mattype, sep='\t', header=0) for idx, val in enumerate(np.arange(1, 28))]
euclid_905mK_omegancdmerrs_1 = [np.sqrt(euclid_905mK_covs_1[idx].iloc[6,6]) for idx, val in enumerate(np.arange(1, 28))]

masses = np.append(np.array([0.001, 0.01]), np.geomspace(0.1, 22.0, 25))
plt.figure(figsize=(15,7.5))
plt.semilogx(masses, np.array(euclid_905mK_omegancdmerrs_1), linestyle='solid', color='red')
plt.title("EUCLID + PLANCK, 0.905[K]")
plt.xlabel("Relic Mass [eV]")
plt.ylabel(r"$\sigma_{\omega_{ncdm}}$")
plt.tick_params(axis='x', which='minttor')
plt.grid(True, which='minor')
plt.savefig("/Users/nicholasdeporzio/Desktop/Plot1.png")
plt.show()

In [ ]:
plt.figure(figsize=(15,7.5))
#plt.semilogx(masses, boss_1500mK_Nerrs, label="BOSS, 1.5072K", linestyle='dotted', color='orange')
#plt.semilogx(masses, boss_1100mK_Nerrs, label="BOSS, 0.9052K", linestyle='dotted', color='green')
plt.semilogx(masses, desi_1500mK_Nerrs, label="DESI, 1.5072K", linestyle='dashed', color='orange')
plt.semilogx(masses, desi_1100mK_Nerrs, label="DESI, 0.9052K", linestyle='dashed', color='green')
#plt.semilogx(masses, euclid_1500mK_Nerrs, label="EUCLID, 1.5072K", color='orange')
#plt.semilogx(masses, euclid_1100mK_Nerrs, label="EUCLID, 0.9052K", color='green')
plt.title(r"$N_{eff}$ Shift, LSS + CMBS4")
plt.xlabel("Relic Mass [eV]")
plt.ylabel(r"$\Delta N_{eff}$")
plt.legend()
plt.tick_params(axis='x', which='minor')
plt.grid(True, which='minor')
plt.savefig("/Users/nicholasdeporzio/Desktop/relicforecast_N.png")
plt.show() 

In [ ]:
print(boss_1500mK_Nerrs[0])
print(boss_1100mK_Nerrs[0])
print(desi_1500mK_Nerrs[0])
print(desi_1100mK_Nerrs[0])
print(euclid_1500mK_Nerrs[0])
print(euclid_1100mK_Nerrs[0])

In [ ]:
errors_Tncdm_cmb = [np.sqrt(fval.numpy_cmb_covariance[6,6]) for fidx, fval in enumerate(forecastset)]    
errors_Tncdm_lss = [np.sqrt(fval.numpy_lss_covariance[6,6]) for fidx, fval in enumerate(forecastset)]
errors_Tncdm_full = [np.sqrt(fval.numpy_full_covariance[6,6]) for fidx, fval in enumerate(forecastset)]

# Plot Set 11 - Demonstrate Shift Formalism in b0
# GOOD

In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import seaborn as sns 
import cosmicfish as cf 
import matplotlib.pyplot as plt
import dill


# Instruct pyplot to use seaborn 
sns.set()

# Set project, data, CLASS directories 
projectdir = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/")
datastore = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/data.nosync/")
classpath = os.path.join(projectdir, "class")

# Specify resolution of numerical integrals
derivative_step = 0.008 # How much to vary parameter to calculate numerical derivative
mu_integral_step = 0.05 # For calculating numerical integral wrt mu between -1 and 1 

# Generate output paths  
ps11_resultsdir = os.path.join(projectdir, 'results', 'ps11')
ps11_convergencedir = os.path.join(ps11_resultsdir, 'convergence')
cf.makedirectory(ps11_resultsdir)
cf.makedirectory(ps11_convergencedir) 

# Set fiducial cosmology
ps11_fid = { # Fiducial for DESI/EUCLID MCMCs
        "A_s" : 2.22e-9, 
        "n_s" : 0.965,
        "omega_b" : 0.02222,
        "omega_cdm" : 0.1120,
        "tau_reio" : 0.06,
        "h" : 0.70,
        "T_cmb" : 2.726, # Units [K]
        "N_ncdm" : 3., 
        "T_ncdm" : 1.95/2.726, 
        "m_ncdm" : 0.03, # Units [eV]
        "b0" : 1.0, 
        "alphak2" : 1.0,
        "sigma_fog_0" : 250000, #Units [m s^-2]
        "N_eff" : 3.046 - (3 * 1.0132), 
        "relic_fix" : None, #Not used for neutrino forecasts
        } 

# DESI survey parameters
z_table = np.array([0.65, 0.75, 0.85, 0.95, 1.05, 1.15, 1.25, 1.35, 1.45, 1.55, 1.65, 1.75, 1.85])
dNdz = np.array([309., 2269., 1923., 2094., 1441., 1353., 1337., 523., 466., 329., 126., 0., 0.])
skycover = 14000. # Sky coverage of survey in degrees^2

# Run Forecast
ps11_forecast = cf.forecast(
    classpath, 
    datastore, 
    'neutrino', 
    ps11_fid, 
    z_table, 
    "DESI"
    dNdz, 
    fcoverage_deg=skycover, 
    dstep=derivative_step,
    RSD=True,
    FOG=True,
    AP=True,
    COV=True)
ps11_forecast.gen_pm()
ps11_forecast.gen_fisher(
    fisher_order=[
        'omega_b',                                    
        'omega_cdm',                                  
        'n_s',                                        
        'A_s',                                        
        'tau_reio',                          
        'h',                                          
        'M_ncdm',                                                                    
        'sigma_fog',                                   
        'b0',                                         
        'alpha_k2',
        'b_Amp'
    ],
    mu_step=mu_integral_step, 
    skipgen=False)

# Restructure results
DD = np.array(ps11_forecast.fisher[10, 10])
Dj = np.array(ps11_forecast.fisher[0:10, 10])
Extended_Fisher = np.array(ps11_forecast.fisher)
ps11_forecast.fisher = np.array(ps11_forecast.fisher[0:10, 0:10])
ps11_forecast.fisher_order.remove('b_Amp')

# Add CMB Data
ps11_forecast.load_cmb_fisher(
    fisher_order=[
        'omega_b',                                    
        'omega_cdm',                                  
        'n_s',                                        
        'A_s',                                        
        'tau_reio',                                   
        'h',                                                                             
        'M_ncdm'],
    fisherpath=os.path.join(cf.priors_directory(), "CMBS4_Fisher_Neutrinos.dat"))

# Save results
ps11_forecast.export_matrices(ps11_resultsdir)
pd.DataFrame(ps11_forecast.numpy_full_fisher, columns=[
        'omega_b',                                    
        'omega_cdm',                                  
        'n_s',                                        
        'A_s',                                        
        'tau_reio',                          
        'h',                                          
        'M_ncdm',                                                                    
        'sigma_fog',                                   
        'b0',                                         
        'alpha_k2'
    ]).to_csv(os.path.join(ps11_resultsdir, "yescmbfisher.csv"), sep='\t')
pd.DataFrame(ps11_forecast.numpy_lss_fisher, columns=[
        'omega_b',                                    
        'omega_cdm',                                  
        'n_s',                                        
        'A_s',                                        
        'tau_reio',                          
        'h',                                          
        'M_ncdm',                                                                    
        'sigma_fog',                                   
        'b0',                                         
        'alpha_k2'
    ]).to_csv(os.path.join(ps11_resultsdir, "nocmbfisher.csv"), sep='\t')

# Print the uncertainty in the P_g shift amplitude
D_spread = np.sqrt(DD)
np.savetxt(path+"shift_root_DD.csv", np.array([D_spread]), delimiter='\t')
print(r"Uncertainty in P_g shift amplitude: ", D_spread)

#Djvec1=(ps11_forecast.numpy_full_fisher[8,:]  *  0.001)

# Print parameter errors
for i in range(10): 
    print(np.sqrt(ps11_forecast.numpy_full_covariance[i, i]), '\n')

# Print shift amplitude
#Amp_Projection[4] = 0.
pd.DataFrame(np.array(Dj)).to_csv(os.path.join(ps11_resultsdir, "shift_basis_projection.csv"), sep='\t')
print("Amplitude vector, <D, g_i>:")
print(pd.DataFrame(np.array(Dj)))

# Print parameter shift  
Delta_A = np.matmul(np.linalg.inv(np.array(ps11_forecast.numpy_full_fisher)), Dj)
print('Parameter shift vector, delta a_i: ')
pd.DataFrame(np.array(Delta_A)).to_csv(os.path.join(ps11_resultsdir, "shift_parameter_amplitude.csv"), sep='\t')
print(pd.DataFrame(np.array(Delta_A)))
print("Parameter shift/error vector, (delta a_i / sigma_i): ")
params = [
        'omega_b',                                    
        'omega_cdm',                                  
        'n_s',                                        
        'A_s',                                        
        'tau_reio',                                   
        'h',                                          
        'M_ncdm',                                                                    
        'sigma_fog',                                   
        'b0',                                         
        'alpha_k2'
    ]
for i in range(10): 
    print("delta a_i / sigma_i for", params[i], ": ", (Delta_A[i]
                                                       /np.sqrt(np.linalg.inv(np.array(ps11_forecast.numpy_full_fisher))[i,i])))
dill.dump_session(os.path.join(ps11_resultsdir, 'ps11.db'))

# Plot Set 12 - Forecast Shift in Neutrino Cosmology w/ and w/o RelicFast
# GOOD



In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import seaborn as sns 
import cosmicfish as cf 
import matplotlib.pyplot as plt
import dill


# Instruct pyplot to use seaborn 
sns.set()

# Set project, data, CLASS directories 
projectdir = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/")
datastore = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/data.nosync12/")
classpath = os.path.join(projectdir, "class")

# Specify resolution of numerical integrals
derivative_step = 0.008 # How much to vary parameter to calculate numerical derivative
mu_integral_step = 0.05 # For calculating numerical integral wrt mu between -1 and 1 

# Generate output paths  
ps12_resultsdir = os.path.join(projectdir, 'results', 'ps12')
ps12_convergencedir = os.path.join(ps12_resultsdir, 'convergence')
cf.makedirectory(ps12_resultsdir)
cf.makedirectory(ps12_convergencedir) 

# Set fiducial cosmology
#ps12_fid = { # Fiducial for DESI/EUCLID MCMCs
#       "A_s" : 2.22e-9, 
#        "n_s" : 0.965,
#        "omega_b" : 0.02222,
#        "omega_cdm" : 0.1120,
#        "tau_reio" : 0.06,
#        "h" : 0.70,
#        "T_cmb" : 2.726, # Units [K]
#        "N_ncdm" : 3., 
#        "T_ncdm" : 1.95/2.726, 
#        "m_ncdm" : 0.03, # Units [eV]
#        "b0" : 1.0, 
#        "alphak2" : 1.0,
#        "sigma_fog_0" : 250000, #Units [m s^-2]
#        "N_eff" : 3.046 - (3 * 1.0132), 
#        "relic_fix" : None, #Not used for neutrino forecasts
#        } 
# Best fit to No Relicfast CMBS4_euclid_beta_3_DH chains
#ps12_fid = { 
#        "A_s" : 2.242754e-09, 
#        "n_s" : 9.669822e-01,
#        "omega_b" : 2.225199e-02,
#        "omega_cdm" : 1.125597e-01,
#        "tau_reio" : 6.233920e-02,
#        "h" : 7.005455e-01,
#        "T_cmb" : 2.726, # Units [K]
#        "N_ncdm" : 3., 
#        "T_ncdm" : 1.95/2.726, 
#        "m_ncdm" : 3.891306e-02, # Units [eV]
#        "b0" : 1.0, 
#        "beta0" : 1.707003e+00, 
#        "beta1" : 0.5,
#        "alphak2" : 9.945805e-01,
#        "sigma_fog_0" : 2.497794e+05, #Units [m s^-2]
#        "N_eff" : 3.046 - (3 * 1.0132), 
#        "relic_fix" : None, #Not used for neutrino forecasts
#        }

# Best fit to No Relicfast CMBS4_euclid_beta_2_DH chains
#ps12_fid = { 
#        "A_s" : 2.238288e-09, 
#        "n_s" : 9.668334e-01,
#        "omega_b" : 2.225588e-02,
#        "omega_cdm" : 1.126466e-01,
#        "tau_reio" : 6.127119e-02,
#        "h" : 7.007553e-01,
#        "T_cmb" : 2.726, # Units [K]
#        "N_ncdm" : 3., 
#        "T_ncdm" : 1.95/2.726, 
#        "m_ncdm" : 1.127821e-01/3, # Units [eV]
#        "b0" : 1.0, 
#        "beta0" : 1.705295e+00, 
#        "beta1" : 1.002319e+00,
#        "alphak2" : 9.909839e-01,
#        "sigma_fog_0" : 2.498509e+05, #Units [m s^-2]
#        "N_eff" : 3.046 - (3 * 1.0132), 
#        "relic_fix" : None, #Not used for neutrino forecasts
#        }

# No Relicfast CMBS4_desi_DH 2020-01-30 Chains Bestfit 
ps12_fid = { 
        "A_s" : 2.234591e-09, 
        "n_s" : 9.659302e-01,
        "omega_b" : 2.227047e-02,
        "omega_cdm" : 1.127401e-01,
        "tau_reio" : 6.019503e-02,
        "h" : 7.007968e-01,
        "T_cmb" : 2.726, # Units [K]
        "N_ncdm" : 3., 
        "T_ncdm" : 1.95/2.726, 
        "m_ncdm" : 3.623808e-02, # Units [eV]
        "b0" : 1.003223e+00, 
        "alphak2" : 9.948011e-01,
        "sigma_fog_0" : 2.497568e+05, #Units [m s^-2]
        "N_eff" : 3.046 - (3 * 1.0132), 
        "relic_fix" : None, #Not used for neutrino forecasts
        }

# DESI survey parameters
z_table = np.array([0.65, 0.75, 0.85, 0.95, 1.05, 1.15, 1.25, 1.35, 1.45, 1.55, 1.65, 1.75, 1.85])
dNdz = np.array([309., 2269., 1923., 2094., 1441., 1353., 1337., 523., 466., 329., 126., 0., 0.])
skycover = 14000. # Sky coverage of survey in degrees^2

# EUCLID values
#z_table = np.array([0.65, 0.75, 0.85, 0.95, 1.05, 1.15, 1.25, 1.35, 1.45, 1.55, 1.65, 1.75, 1.85, 1.95])
#dNdz = np.array([2434.280, 4364.812, 4728.559, 4825.798, 4728.797, 4507.625, 4269.851, 3720.657, 3104.309, 
#    2308.975, 1514.831, 1474.707, 893.716, 497.613])
#skycover = 0.3636 # Sky coverage of survey in fraction

# Run Forecast
ps12_forecast = cf.forecast(
    classpath, 
    datastore, 
    'neutrino', 
    ps12_fid, 
    z_table, 
    "DESI",
    dNdz, 
    fcoverage_deg=skycover, 
    dstep=derivative_step,
    RSD=True,
    FOG=True,
    AP=True,
    COV=True)
ps12_forecast.gen_pm()
ps12_forecast.gen_fisher(
    fisher_order=[
        'omega_b',                                    
        'omega_cdm',                                  
        'n_s',                                        
        'A_s',                                        
        'tau_reio',                                   
        'h',                                          
        'M_ncdm',                                                                    
        'sigma_fog',                                   
        'b0',                                      
        'alpha_k2',
        'D_Amp'
    ],
    mu_step=mu_integral_step, 
    skipgen=False)


In [ ]:
# Restructure results
DD = np.array(ps12_forecast.fisher[10, 10])
Dj = np.array(ps12_forecast.fisher[0:10, 10])
Extended_Fisher = np.array(ps12_forecast.fisher)
ps12_forecast.fisher = np.array(ps12_forecast.fisher[0:10, 0:10])
ps12_forecast.fisher_order.remove('D_Amp')

# Save notebook state
dill.dump_session(os.path.join(ps12_resultsdir, 'ps12.db'))

# Add CMB Data
ps12_forecast.load_cmb_fisher(
    fisher_order=[
        'omega_b',                                    
        'omega_cdm',                                  
        'n_s',                                        
        'A_s',                                        
        'tau_reio',                                   
        'h',                                                                             
        'M_ncdm'],
    fisherpath=os.path.join(cf.priors_directory(), "CMBS4_Fisher_Neutrinos.dat"))

# Save results
ps12_forecast.export_matrices(ps12_resultsdir)
pd.DataFrame(ps12_forecast.numpy_full_fisher, columns=[
        'omega_b',                                    
        'omega_cdm',                                  
        'n_s',                                        
        'A_s',                                        
        'tau_reio',                                   
        'h',                                          
        'M_ncdm',                                                                    
        'sigma_fog',                                   
        'b0',                                    
        'alpha_k2'
    ]).to_csv(os.path.join(ps12_resultsdir, "yescmbfisher.csv"), sep='\t')
pd.DataFrame(ps12_forecast.numpy_lss_fisher, columns=[
        'omega_b',                                    
        'omega_cdm',                                  
        'n_s',                                        
        'A_s',                                        
        'tau_reio',                                   
        'h',                                          
        'M_ncdm',                                                                    
        'sigma_fog',                                   
        'b0',                                     
        'alpha_k2'
    ]).to_csv(os.path.join(ps12_resultsdir, "nocmbfisher.csv"), sep='\t')

# Print the uncertainty in the P_g shift amplitude
D_spread = np.sqrt(DD)
np.savetxt(os.path.join(ps12_resultsdir, "shift_root_DD.csv"), np.array([D_spread]), delimiter='\t')
print(r"Uncertainty in P_g shift amplitude: ", D_spread)

#Djvec1=(ps12_forecast.numpy_full_fisher[8,:]  *  0.001)

# Print parameter errors
for i in range(10): 
    print(np.sqrt(ps12_forecast.numpy_full_covariance[i, i]), '\n')

# Print shift amplitude
#Amp_Projection[4] = 0.
pd.DataFrame(np.array(Dj)).to_csv(os.path.join(ps12_resultsdir, "shift_basis_projection.csv"), sep='\t')
print("Amplitude vector, <D, g_i>:")
print(pd.DataFrame(np.array(Dj)))

# Print parameter shift  
Delta_A = np.matmul(np.linalg.inv(np.array(ps12_forecast.numpy_full_fisher)), Dj)
print('Parameter shift vector, delta a_i: ')
pd.DataFrame(np.array(Delta_A)).to_csv(os.path.join(ps12_resultsdir, "shift_parameter_amplitude.csv"), sep='\t')
print(pd.DataFrame(np.array(Delta_A)))
print("Parameter shift/error vector, (delta a_i / sigma_i): ")

In [ ]:
params = [
        'omega_b',                                    
        'omega_cdm',                                  
        'n_s',                                        
        'A_s',                                        
        'tau_reio',                                   
        'h',                                          
        'M_ncdm',                                                                    
        'sigma_fog',                                   
        'b0',
        'alpha_k2'
    ]
for i in range(10): 
    print("delta a_i / sigma_i for", params[i], ": ", (Delta_A[i]
                                                       /np.sqrt(np.linalg.inv(np.array(ps12_forecast.numpy_full_fisher))[i,i])))



# Save Datasets

In [ ]:
varnames = ['A_s', 'n_s', 'omega_b', 'omega_cdm', 'tau_reio', 'h', 'T_ncdm']
for fidx, fval  in enumerate(ps6_forecastset): 
    mass = ps6_fiducialset[fidx]['m_ncdm']
    omega_b_high = fval.omega_b_high[0].datadirectory
    omega_b_low = fval.omega_b_low[0].datadirectory
    omega_cdm_high = fval.omega_cdm_high[0].datadirectory
    omega_cdm_low = fval.omega_cdm_low[0].datadirectory
    h_high = fval.h_high[0].datadirectory
    h_low = fval.h_low[0].datadirectory
    tau_reio_high = fval.tau_reio_high[0].datadirectory
    tau_reio_low = fval.tau_reio_low[0].datadirectory
    T_ncdm_high = fval.T_ncdm_high[0].datadirectory
    T_ncdm_low = fval.T_ncdm_low[0].datadirectory
    A_s_high = fval.A_s_high[0].datadirectory
    A_s_low = fval.A_s_low[0].datadirectory
    n_s_high = fval.n_s_high[0].datadirectory
    n_s_low = fval.n_s_low[0].datadirectory
    for pidx, pval  in enumerate(varnames): 
        #path = ('/Users/nicholasdeporzio/Desktop/Relic_1507mK_Cls/m_ncdm_' + str(mass) + '/' + pval + '/')
        #shutil.copytree(eval((pval+'_high')), (path+'high'))
        #shutil.copytree(eval((pval+'_low')), (path+'low'))
        path = ('/Users/nicholasdeporzio/Desktop/Relic_1507mK_Cls/Unsorted/')
        shutil.move(eval((pval+'_high')), path)
        shutil.move(eval((pval+'_low')), path)

In [ ]:
varnames = ['A_s', 'n_s', 'omega_b', 'omega_cdm', 'tau_reio', 'h', 'T_ncdm']
for fidx, fval  in enumerate(ps5_forecastset): 
    mass = ps5_fiducialset[fidx]['m_ncdm']
    omega_b_high = fval.omega_b_high[0].datadirectory
    omega_b_low = fval.omega_b_low[0].datadirectory
    omega_cdm_high = fval.omega_cdm_high[0].datadirectory
    omega_cdm_low = fval.omega_cdm_low[0].datadirectory
    h_high = fval.h_high[0].datadirectory
    h_low = fval.h_low[0].datadirectory
    tau_reio_high = fval.tau_reio_high[0].datadirectory
    tau_reio_low = fval.tau_reio_low[0].datadirectory
    T_ncdm_high = fval.T_ncdm_high[0].datadirectory
    T_ncdm_low = fval.T_ncdm_low[0].datadirectory
    A_s_high = fval.A_s_high[0].datadirectory
    A_s_low = fval.A_s_low[0].datadirectory
    n_s_high = fval.n_s_high[0].datadirectory
    n_s_low = fval.n_s_low[0].datadirectory
    for pidx, pval  in enumerate(varnames): 
        path = ('/Users/nicholasdeporzio/Desktop/desi_1.1k_cls/m_ncdm_' + str(mass) + '/' + pval + '/')
        shutil.copytree(eval((pval+'_high')), (path+'high'))
        shutil.copytree(eval((pval+'_low')), (path+'low'))